# Đọc dữ liệu BCSVH các nhà máy TĐN từ DGMS

### 1. Miền Bắc

In [1]:
import pandas as pd
import os
import re
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

def read_and_process_a1_npc_data(root_path=r'P:/15. Hệ thống DGMS/Daily Report/Bac/BCSVH_xlsx'):
    """
    Đọc và xử lý dữ liệu A1/NPC thành một luồng thống nhất:
    1. Đọc dữ liệu từ file A1 và NPC
    2. Lọc theo "Báo cáo sau vận hành"
    3. Xử lý cấu trúc cột
    4. Trả về kết quả cuối cùng
    """
    
    # Danh sách cột mong muốn theo đúng thứ tự
    expected_columns = [
        'Trạng thái', 'Code', 'Nhà máy', 'P.đặt', 'Q.ĐK', 'Điện lực', 'Loại',
        '00:30', '01:00', '01:30', '02:00', '02:30', '03:00', '03:30', '04:00', '04:30',
        '05:00', '05:30', '06:00', '06:30', '07:00', '07:30', '08:00', '08:30', '09:00',
        '09:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30',
        '14:00', '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00',
        '18:30', '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:00', '22:30',
        '23:00', '23:30', '00:00'
    ]
    
    df_list = []
    total_files = 0
    successful_files = 0
    failed_files = 0
    files_by_date = {}
    
    print(f"🚀 Bắt đầu đọc và xử lý dữ liệu A1/NPC từ: {root_path}")
    
    # BƯỚC 1: ĐỌC DỮ LIỆU
    for year in os.listdir(root_path):
        year_path = os.path.join(root_path, year)
        if not os.path.isdir(year_path):
            continue
        
        print(f"\n📅 Đang xử lý năm: {year}")
        
        for month in range(1, 13):
            folder_path = os.path.join(year_path, f'{month:02d}')
            
            if not os.path.exists(folder_path):
                continue
            
            print(f"  📂 Tháng {month:02d}...")
            month_files = 0
            
            for file in os.listdir(folder_path):
                if file.endswith('.xlsx'):
                    if '-A1' not in file and '-NPC' not in file:
                        continue
                        
                    total_files += 1
                    date_match = re.search(r'(\d{4}\.\d{2}\.\d{2})', file)
                    date_part = date_match.group(1) if date_match else ""
                    file_path = os.path.join(folder_path, file)
                    
                    try:
                        if '-A1' in file:
                            file_type = 'A1'
                        elif '-NPC' in file:
                            file_type = 'NPC'
                        
                        if date_part:
                            if date_part not in files_by_date:
                                files_by_date[date_part] = set()
                            files_by_date[date_part].add(file_type)
                        
                        df_full = pd.read_excel(file_path, header=None)
                        
                        # Tìm vị trí cột "Trạng thái"
                        start_row, status_col = None, None
                        found = False
                        
                        for r_idx, row in df_full.iterrows():
                            for c_idx, cell_value in enumerate(row):
                                if str(cell_value).strip() == 'Trạng thái':
                                    start_row = r_idx
                                    status_col = c_idx
                                    found = True
                                    break
                            if found:
                                break
                        
                        if start_row is not None:
                            header_row = df_full.iloc[start_row]
                            
                            # Tìm vị trí cột "00:00"
                            end_col = None
                            for c_idx, cell_value in enumerate(header_row):
                                if str(cell_value).strip() == '00:00':
                                    end_col = c_idx
                                    break
                            
                            if end_col is not None:
                                data_cols = list(range(status_col, end_col + 1))
                                actual_header = [str(header_row.iloc[i]).strip() for i in data_cols]
                                df_data = df_full.iloc[start_row + 1:, data_cols].copy()
                                
                                df_data.columns = actual_header
                                df_data.dropna(how='all', inplace=True)
                                df_data.reset_index(drop=True, inplace=True)
                                
                                if len(actual_header) >= len(expected_columns):
                                    df_data = df_data.iloc[:, :len(expected_columns)]
                                    df_data.columns = expected_columns
                                    
                                    # Thêm thông tin meta
                                    df_data.insert(0, 'Ngày', date_part)
                                    df_data.insert(1, 'Loại_File', file_type)
                                    
                                    df_list.append(df_data)
                                    successful_files += 1
                                    month_files += 1
                                    
                                    if total_files % 20 == 0:
                                        print(f"    ✅ Đã xử lý {total_files} file...")
                                else:
                                    failed_files += 1
                            else:
                                failed_files += 1
                        else:
                            failed_files += 1
                            
                    except Exception as e:
                        print(f"    ❌ Lỗi file {file}: {e}")
                        failed_files += 1
            
            if month_files > 0:
                print(f"    ✅ Hoàn thành tháng {month:02d}: {month_files} file")
    
    # Kiểm tra file thiếu
    check_missing_a1_npc_files(files_by_date)
    
    if not df_list:
        print(f'\n❌ Không đọc được dữ liệu A1/NPC nào')
        return pd.DataFrame()
    
    # Gộp tất cả dữ liệu
    df_raw = pd.concat(df_list, ignore_index=True)
    print(f'\n📊 ĐỌC DỮ LIỆU THÀNH CÔNG:')
    print(f'   - Tổng file A1/NPC kiểm tra: {total_files}')
    print(f'   - File đọc thành công: {successful_files}')
    print(f'   - File lỗi: {failed_files}')
    print(f'   - Tổng số dòng dữ liệu thô: {len(df_raw):,}')
    
    # BƯỚC 2: XỬ LÝ VÀ LỌC DỮ LIỆU
    print(f'\n🔄 ĐANG XỬ LÝ VÀ LỌC DỮ LIỆU...')
    
    # Lọc chỉ lấy "Báo cáo sau vận hành"
    df_filtered = df_raw[df_raw['Loại'].isin(['Báo cáo sau vận hành', 'Báo cáo sau vận hành tổng hợp'])]
    print(f'   - Sau khi lọc theo loại báo cáo: {len(df_filtered):,} dòng')
    
    if df_filtered.empty:
        print(f'❌ Không có dữ liệu sau khi lọc!')
        return pd.DataFrame()
    
    # Loại bỏ các cột không cần thiết
    df_processed = df_filtered.drop(columns=['Code', 'Loại', 'Trạng thái']).reset_index(drop=True)
    print(f'   - Sau khi loại bỏ cột không cần thiết: {len(df_processed.columns)} cột')
    
    # Sắp xếp lại cột: Ngày đặt trước cột 00:30
    if 'Ngày' in df_processed.columns and '00:30' in df_processed.columns:
        cols = df_processed.columns.tolist()
        cols.remove('Ngày')
        time_30_index = cols.index('00:30')
        cols.insert(time_30_index, 'Ngày')
        df_processed = df_processed[cols]
        print(f'   - Đã sắp xếp lại thứ tự cột')
    
    # BƯỚC 3: KẾT QUẢ CUỐI CÙNG
    print(f'\n🎉 HOÀN THÀNH XỬ LÝ DỮ LIỆU:')
    print(f'   - Tổng số dòng cuối cùng: {len(df_processed):,}')
    print(f'   - Số cột cuối cùng: {len(df_processed.columns)}')
    
    # Hiển thị thống kê
    if 'Loại_File' in df_processed.columns:
        print(f'\n📁 Thống kê theo loại file:')
        file_stats = df_processed['Loại_File'].value_counts()
        for file_type, count in file_stats.items():
            print(f'   - {file_type}: {count:,} dòng')
    
    if 'Ngày' in df_processed.columns:
        print(f'\n📆 Khoảng thời gian:')
        dates = pd.to_datetime(df_processed['Ngày'], format='%Y.%m.%d', errors='coerce')
        dates = dates.dropna()
        if not dates.empty:
            print(f'   - Từ: {dates.min().strftime("%Y.%m.%d")}')
            print(f'   - Đến: {dates.max().strftime("%Y.%m.%d")}')
    
    print(f'\n📋 CẤU TRÚC CỘT CUỐI CÙNG:')
    for i, col in enumerate(df_processed.columns, 1):
        print(f'   {i:2d}. {col}')
    
    return df_processed

def read_and_process_a1_npc_by_date_range(start_date_str, end_date_str, root_path=r'P:/15. Hệ thống DGMS/Daily Report/Bac/BCSVH_xlsx'):
    """
    Đọc và xử lý dữ liệu A1/NPC theo khoảng ngày - luồng thống nhất
    """
    
    try:
        start_date = datetime.strptime(start_date_str, '%Y%m%d')
        end_date = datetime.strptime(end_date_str, '%Y%m%d')
        
        if start_date > end_date:
            print("❌ Ngày bắt đầu không thể lớn hơn ngày kết thúc!")
            return pd.DataFrame()
            
    except ValueError:
        print("❌ Định dạng ngày không đúng! Vui lòng sử dụng format YYYYMMDD")
        return pd.DataFrame()
    
    expected_columns = [
        'Trạng thái', 'Code', 'Nhà máy', 'P.đặt', 'Q.ĐK', 'Điện lực', 'Loại',
        '00:30', '01:00', '01:30', '02:00', '02:30', '03:00', '03:30', '04:00', '04:30',
        '05:00', '05:30', '06:00', '06:30', '07:00', '07:30', '08:00', '08:30', '09:00',
        '09:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30',
        '14:00', '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00',
        '18:30', '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:00', '22:30',
        '23:00', '23:30', '00:00'
    ]
    
    df_list = []
    total_files = 0
    successful_files = 0
    files_by_date = {}
    
    # Tạo danh sách ngày target
    current_date = start_date
    target_dates = set()
    
    while current_date <= end_date:
        date_str = current_date.strftime('%Y.%m.%d')
        target_dates.add(date_str)
        files_by_date[date_str] = set()
        current_date += timedelta(days=1)
    
    print(f"🚀 Đọc và xử lý dữ liệu A1/NPC từ {start_date.strftime('%Y.%m.%d')} đến {end_date.strftime('%Y.%m.%d')}")
    print(f"📊 Tổng cộng {len(target_dates)} ngày cần xử lý")
    
    # BƯỚC 1: ĐỌC DỮ LIỆU THEO KHOẢNG NGÀY
    years_in_range = set()
    for date_str in target_dates:
        year = date_str.split('.')[0]
        years_in_range.add(year)
    
    for year in sorted(years_in_range):
        year_path = os.path.join(root_path, year)
        
        if not os.path.exists(year_path):
            continue
        
        print(f"\n📅 Đang xử lý năm: {year}")
        
        for month in range(1, 13):
            folder_path = os.path.join(year_path, f'{month:02d}')
            
            if not os.path.exists(folder_path):
                continue
            
            for file in os.listdir(folder_path):
                if file.endswith('.xlsx'):
                    if '-A1' not in file and '-NPC' not in file:
                        continue
                        
                    date_match = re.search(r'(\d{4}\.\d{2}\.\d{2})', file)
                    date_part = date_match.group(1) if date_match else ""
                    
                    if date_part not in target_dates:
                        continue
                    
                    total_files += 1
                    file_path = os.path.join(folder_path, file)
                    
                    try:
                        if '-A1' in file:
                            file_type = 'A1'
                        elif '-NPC' in file:
                            file_type = 'NPC'
                        
                        files_by_date[date_part].add(file_type)
                        
                        df_full = pd.read_excel(file_path, header=None)
                        
                        start_row, status_col = None, None
                        found = False
                        
                        for r_idx, row in df_full.iterrows():
                            for c_idx, cell_value in enumerate(row):
                                if str(cell_value).strip() == 'Trạng thái':
                                    start_row = r_idx
                                    status_col = c_idx
                                    found = True
                                    break
                            if found:
                                break
                        
                        if start_row is not None:
                            header_row = df_full.iloc[start_row]
                            
                            end_col = None
                            for c_idx, cell_value in enumerate(header_row):
                                if str(cell_value).strip() == '00:00':
                                    end_col = c_idx
                                    break
                            
                            if end_col is not None:
                                data_cols = list(range(status_col, end_col + 1))
                                actual_header = [str(header_row.iloc[i]).strip() for i in data_cols]
                                df_data = df_full.iloc[start_row + 1:, data_cols].copy()
                                
                                df_data.columns = actual_header
                                df_data.dropna(how='all', inplace=True)
                                df_data.reset_index(drop=True, inplace=True)
                                
                                if len(actual_header) >= len(expected_columns):
                                    df_data = df_data.iloc[:, :len(expected_columns)]
                                    df_data.columns = expected_columns
                                    
                                    df_data.insert(0, 'Ngày', date_part)
                                    df_data.insert(1, 'Loại_File', file_type)
                                    
                                    df_list.append(df_data)
                                    successful_files += 1
                                    
                                    if total_files % 10 == 0:
                                        print(f"    ✅ Đã xử lý {total_files} file...")
                        
                    except Exception as e:
                        print(f"    ❌ Lỗi file {file}: {e}")
    
    # Kiểm tra file thiếu
    check_missing_a1_npc_files(files_by_date, target_dates)
    
    if not df_list:
        print(f'\n❌ Không đọc được dữ liệu A1/NPC nào trong khoảng ngày đã chọn')
        return pd.DataFrame()
    
    # Gộp dữ liệu thô
    df_raw = pd.concat(df_list, ignore_index=True)
    print(f'\n📊 ĐỌC DỮ LIỆU THÀNH CÔNG:')
    print(f'   - Tổng file A1/NPC kiểm tra: {total_files}')
    print(f'   - File đọc thành công: {successful_files}')
    print(f'   - Tổng số dòng dữ liệu thô: {len(df_raw):,}')
    
    # BƯỚC 2: XỬ LÝ VÀ LỌC DỮ LIỆU
    print(f'\n🔄 ĐANG XỬ LÝ VÀ LỌC DỮ LIỆU...')
    
    df_filtered = df_raw[df_raw['Loại'].isin(['Báo cáo sau vận hành', 'Báo cáo sau vận hành tổng hợp'])]
    print(f'   - Sau khi lọc theo loại báo cáo: {len(df_filtered):,} dòng')
    
    if df_filtered.empty:
        print(f'❌ Không có dữ liệu sau khi lọc!')
        return pd.DataFrame()
    
    df_processed = df_filtered.drop(columns=['Code', 'Loại', 'Trạng thái']).reset_index(drop=True)
    print(f'   - Sau khi loại bỏ cột không cần thiết: {len(df_processed.columns)} cột')
    
    # Sắp xếp lại cột
    if 'Ngày' in df_processed.columns and '00:30' in df_processed.columns:
        cols = df_processed.columns.tolist()
        cols.remove('Ngày')
        time_30_index = cols.index('00:30')
        cols.insert(time_30_index, 'Ngày')
        df_processed = df_processed[cols]
        print(f'   - Đã sắp xếp lại thứ tự cột')
    
    # Kết quả cuối cùng
    print(f'\n🎉 HOÀN THÀNH XỬ LÝ DỮ LIỆU:')
    print(f'   - Tổng số dòng cuối cùng: {len(df_processed):,}')
    print(f'   - Số cột cuối cùng: {len(df_processed.columns)}')
    
    return df_processed

def check_missing_a1_npc_files(files_by_date, target_dates=None):
    """Kiểm tra file A1 và NPC thiếu cho từng ngày"""
    
    required_files = {'A1', 'NPC'}
    missing_reports = []
    complete_days = 0
    
    dates_to_check = target_dates if target_dates else files_by_date.keys()
    
    for date in sorted(dates_to_check):
        available_files = files_by_date.get(date, set())
        missing_files = required_files - available_files
        
        if not missing_files:
            complete_days += 1
        elif len(missing_files) == 2:
            missing_reports.append(f"   📅 {date}: ❌ Thiếu cả A1 và NPC")
        else:
            missing_file = list(missing_files)[0]
            missing_reports.append(f"   📅 {date}: ⚠️ Thiếu file {missing_file}")
    
    print(f'\n📋 BÁO CÁO FILE A1/NPC THIẾU:')
    total_days = len(dates_to_check)
    missing_days = total_days - complete_days
    
    print(f'   - Tổng số ngày kiểm tra: {total_days}')
    print(f'   - Ngày đầy đủ file (A1 + NPC): {complete_days}')
    print(f'   - Ngày thiếu file: {missing_days}')
    
    if missing_reports:
        print(f'\n⚠️ CHI TIẾT NGÀY THIẾU FILE:')
        for report in missing_reports:
            print(report)
    else:
        print(f'\n✅ Tất cả ngày đều có đầy đủ file A1 và NPC!')

# ===== SỬ DỤNG LUỒNG THỐNG NHẤT =====

print("🔥 LUỒNG THỐNG NHẤT: Đọc và xử lý dữ liệu A1/NPC!")
user_confirm = input("Bạn có muốn đọc tất cả dữ liệu A1/NPC? (y/n): ")

if user_confirm.lower() == 'y':
    # Đọc và xử lý tất cả dữ liệu
    df_bac_final = read_and_process_a1_npc_data()
    
    if not df_bac_final.empty:
        print(f"\n✅ Dữ liệu đã sẵn sàng sử dụng: df_bac_final")
        print(f"Shape: {df_bac_final.shape}")

else:
    print("\n📅 Nhập khoảng ngày cần đọc và xử lý dữ liệu A1/NPC:")
    
    # Nhập ngày bắt đầu
    while True:
        start_date_input = input("Ngày bắt đầu (YYYYMMDD): ").strip()
        try:
            datetime.strptime(start_date_input, '%Y%m%d')
            break
        except ValueError:
            print("❌ Định dạng không đúng! Vui lòng nhập theo format YYYYMMDD")
    
    # Nhập ngày kết thúc
    while True:
        end_date_input = input("Ngày kết thúc (YYYYMMDD): ").strip()
        try:
            end_date = datetime.strptime(end_date_input, '%Y%m%d')
            start_date = datetime.strptime(start_date_input, '%Y%m%d')
            
            if start_date <= end_date:
                break
            else:
                print("❌ Ngày kết thúc phải lớn hơn hoặc bằng ngày bắt đầu!")
        except ValueError:
            print("❌ Định dạng không đúng! Vui lòng nhập theo format YYYYMMDD")
    
    # Đọc và xử lý dữ liệu theo khoảng ngày
    df_bac_final = read_and_process_a1_npc_by_date_range(start_date_input, end_date_input)
    
    if not df_bac_final.empty:
        print(f"\n✅ Dữ liệu đã sẵn sàng sử dụng: df_bac_final")
        print(f"Shape: {df_bac_final.shape}")

print("\n🎯 Hoàn thành luồng thống nhất!")

🔥 LUỒNG THỐNG NHẤT: Đọc và xử lý dữ liệu A1/NPC!
🚀 Bắt đầu đọc và xử lý dữ liệu A1/NPC từ: P:/15. Hệ thống DGMS/Daily Report/Bac/BCSVH_xlsx


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'P:/15. Hệ thống DGMS/Daily Report/Bac/BCSVH_xlsx'

In [ ]:
import os

# Tạo thư mục nếu chưa tồn tại
output_dir_bac = r"D:\OneDrive\0. Code\TDN\data\interim\Bac"
os.makedirs(output_dir_bac, exist_ok=True)

# Lưu dữ liệu ra file parquet
if 'df_bac_final' in locals() and not df_bac_final.empty:
    print("🔧 Đang xử lý kiểu dữ liệu...")

    print(f"Kiểu dữ liệu của cột 'Nhà máy': {df_bac_final['Nhà máy'].dtype}")
    print(f"Các giá trị unique trong cột 'Nhà máy': {df_bac_final['Nhà máy'].unique()[:10]}")

    df_save_bac = df_bac_final.copy()
    df_save_bac['Nhà máy'] = df_save_bac['Nhà máy'].astype(str)

    object_columns_bac = df_save_bac.select_dtypes(include=['object']).columns
    for col in object_columns_bac:
        try:
            df_save_bac[col] = df_save_bac[col].astype(str)
        except:
            print(f"⚠️ Không thể chuyển đổi cột {col}")

    output_file_bac = os.path.join(output_dir_bac, "df_bac_final.parquet")
    df_save_bac.to_parquet(output_file_bac, index=False)
    print(f"✅ Đã lưu dữ liệu ra file: {output_file_bac}")
    print(f"📊 Kích thước file: {os.path.getsize(output_file_bac) / (1024*1024):.2f} MB")
    print(f"📋 Shape: {df_save_bac.shape}")
else:
    print("❌ Không có dữ liệu để lưu!")

In [2]:
df_bac_final = pd.read_parquet(r"D:\OneDrive\0. Code\TDN\data\interim\Bac\df_bac_final.parquet")
df_bac_final

,Loại_File,Nhà máy,P.đặt,Q.ĐK,Điện lực,Ngày,00:30,01:00,01:30,02:00,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,00:00
0,A1,Bản Rạ,18.6,A1,PC_CAOBANG,2024.04.18,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,A1,Bảo Lâm 1,30,A1,PC_CAOBANG,2024.04.18,0,0,0,0,...,19.6,11.4,0,0,0,0,0,0,0,0
2,A1,Bảo Lâm 3A,8,A1,PC_CAOBANG,2024.04.18,0,0,0,0,...,7.5,0,0,0,0,0,0,0,0,0
3,A1,Hòa Thuận,17.4,A1,PC_CAOBANG,2024.04.18,0,0,0,0,...,9.8,10,5.5,0,0,0,0,0,0,0
4,A1,Mông Ân,30,A1,PC_CAOBANG,2024.04.18,0,0,0,0,...,23,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132892,NPC,Ma Lừ Thàng,3,PC_YENBAI,PC_YENBAI,2025.05.31,0,0,0,0,...,3,3,3,0,0,0,0,0,0,0
132893,NPC,Nậm Tục,3,PC_YENBAI,PC_YENBAI,2025.05.31,0.8,0.8,0.8,0.8,...,3,3,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
132894,NPC,Phình Hồ,2.5,PC_YENBAI,PC_YENBAI,2025.05.31,0,0,0,0,...,2.4,2.4,0,0,0,0,0,0,0,0
132895,NPC,Hưng Khánh,0.5,PC_YENBAI,PC_YENBAI,2025.05.31,0.2,0.2,0.2,0.2,...,0.16,0.14,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11


In [7]:
# Đọc dữ liệu tham chiếu
df_thamchieu = pd.read_excel(r"D:\OneDrive\0. Code\TDN\data\raw\DS_TTT_TDN.xlsx")
print(f"✅ Đọc thành công df_thamchieu - Shape: {df_thamchieu.shape}")
df_thamchieu

✅ Đọc thành công df_thamchieu - Shape: (537, 20)


,ID_NM,DIEU_DO,TCTDL,DIEN_LUC,TENNHAMAY,TEN_NM,LOAI_HINH,QUYEN_DIEUKHIEN,CS_DAT,A_TB_NAM,U_DAUNOI,MIEN,TINH,HUYEN,KINHDO,VIDO,BACTHANGSONG,TD_THAMCHIEU,HIEU_LUC,DVPD
0,10997,A1,NPC,PC_BACKAN,Nặm Cắt,NAM_CAT,1,PC_BACKAN,3.2,0.000,35,Bắc,Bắc Kạn,Bạch Thông,105.789306,22.192518,SONG_HONG,TUYEN_QUANG,1,NaN
1,10998,A1,NPC,PC_BACKAN,Pác Cáp,PAC_CAP,1,PC_BACKAN,6.0,0.000,0,NaN,NaN,NaN,106.127303,22.210818,SONG_HONG,TUYEN_QUANG,1,NaN
2,10995,A1,NPC,PC_BACKAN,Tà Làng,TA_LANG,1,PC_BACKAN,4.5,0.000,0,NaN,NaN,NaN,105.721185,22.303931,SONG_HONG,TUYEN_QUANG,1,NaN
3,11193,A1,NPC,PC_BACKAN,THÁC GIỀNG 1,THAC_GIENG 1,1,PC_BACKAN,5.5,0.000,0,NaN,NaN,NaN,105.893735,22.095786,SONG_HONG,TUYEN_QUANG,1,NaN
4,11191,A1,NPC,PC_BACKAN,Thượng Ân,THUONG_AN,1,PC_BACKAN,2.4,0.000,0,NaN,NaN,NaN,106.003945,22.511839,SONG_HONG,TUYEN_QUANG,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,11054,A1,NPC,PC_YENBAI,Thào Sa Chải,THAO_SA_CHAI,1,PC_YENBAI,6.5,22.684,35,Bắc,Yên Bái,Mù Cang Chải,104.268669,21.838929,SONG_HONG,BAN_CHAT,0,NaN
533,10830,A1,NPC,PC_YENBAI,Trạm Tấu,TRAM_TAU,1,A1,30.0,0.000,0,NaN,NaN,NaN,104.442036,21.498660,SONG_HONG,BAN_CHAT,1,NaN
534,10840,A1,NPC,PC_YENBAI,Vực Tuần,VUC_TUAN,1,PC_YENBAI,5.0,0.000,0,NaN,NaN,NaN,104.569095,21.416027,SONG_HONG,BAN_CHAT,1,NaN
535,10837,A1,NPC,PC_YENBAI,Nậm Tục,NAM_TUC,1,PC_YENBAI,3.0,0.000,0,NaN,NaN,NaN,104.447998,21.608201,SONG_HONG,BAN_CHAT,1,NaN


In [8]:
# Merge dữ liệu tham chiếu vào df_bac_final
print("🔄 Đang merge dữ liệu tham chiếu...")

# Kiểm tra các cột key trước khi merge
print(f"📊 Số lượng nhà máy unique trong df_bac_final: {df_bac_final['Nhà máy'].nunique()}")
print(f"📊 Số lượng nhà máy unique trong df_thamchieu: {df_thamchieu['TENNHAMAY'].nunique()}")

# Thực hiện merge với left join để giữ tất cả dữ liệu từ df_bac_final
df_merged_bac = df_bac_final.merge(
    df_thamchieu[['TENNHAMAY', 'TD_THAMCHIEU']], 
    left_on='Nhà máy', 
    right_on='TENNHAMAY', 
    how='left'
)

print(f"✅ Merge hoàn thành - Shape: {df_merged_bac.shape}")
print(f"📋 Số dòng trước merge: {df_bac_final.shape[0]}")
print(f"📋 Số dòng sau merge: {df_merged_bac.shape[0]}")

# Kiểm tra số lượng giá trị null trong cột TD_THAMCHIEU
null_count_bac = df_merged_bac['TD_THAMCHIEU'].isnull().sum()
print(f"⚠️ Số dòng không có thông tin tham chiếu: {null_count_bac}")

# Hiển thị một số dòng mẫu
print("\n📝 Dữ liệu sau merge:")
print(df_merged_bac[['Nhà máy', 'TENNHAMAY', 'TD_THAMCHIEU']].head())

# Cập nhật df_bac_final
df_bac_final = df_merged_bac.drop(columns=['TENNHAMAY'])
print(f"🔄 Đã cập nhật df_bac_final - Shape: {df_bac_final.shape}")

🔄 Đang merge dữ liệu tham chiếu...
📊 Số lượng nhà máy unique trong df_bac_final: 342
📊 Số lượng nhà máy unique trong df_thamchieu: 534


✅ Merge hoàn thành - Shape: (133488, 56)
📋 Số dòng trước merge: 132897
📋 Số dòng sau merge: 133488
⚠️ Số dòng không có thông tin tham chiếu: 10776

📝 Dữ liệu sau merge:
      Nhà máy   TENNHAMAY TD_THAMCHIEU
0      Bản Rạ      Bản Rạ  TUYEN_QUANG
1   Bảo Lâm 1   Bảo Lâm 1  TUYEN_QUANG
2  Bảo Lâm 3A  Bảo Lâm 3A  TUYEN_QUANG
3   Hòa Thuận   Hòa Thuận  TUYEN_QUANG
4     Mông Ân     Mông Ân  TUYEN_QUANG
🔄 Đã cập nhật df_bac_final - Shape: (133488, 55)


In [11]:
cols_to_numeric = ['P.đặt'] + [col for col in df_bac_final.columns if re.match(r'^\d{2}:\d{2}$', str(col))]
for col in cols_to_numeric:
    if col in df_bac_final.columns:
        df_bac_final[col] = pd.to_numeric(df_bac_final[col], errors='coerce')

In [12]:
# Xuất dữ liệu ra file Excel vào thư mục Nam
print("📤 Đang xuất dữ liệu ra file Excel...")

# Tạo tên file với date
from datetime import datetime
date_str = datetime.now().strftime("%Y%m%d")
output_file = f"D:/OneDrive/0. Code/TDN/data/interim/Bac/DGMS_data_{date_str}.xlsx"

# Xuất df_nam_final ra Excel
df_bac_final.to_excel(output_file, index=False)

print(f"✅ Xuất file thành công: {output_file}")
print(f"📊 Dữ liệu đã được lưu với {df_bac_final.shape[0]} dòng và {df_bac_final.shape[1]} cột")

📤 Đang xuất dữ liệu ra file Excel...


✅ Xuất file thành công: D:/OneDrive/0. Code/TDN/data/interim/Bac/DGMS_data_20250619.xlsx
📊 Dữ liệu đã được lưu với 133488 dòng và 55 cột


### 2. Miền Nam

In [ ]:
import pandas as pd
import os
import re
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

def read_and_process_a2_spc_data(root_path=r'P:/15. Hệ thống DGMS/Daily Report/Nam/BCSVH_xlsx'):
    """
    Đọc và xử lý dữ liệu A2/SPC thành một luồng thống nhất:
    1. Đọc dữ liệu từ file A2 và SPC
    2. Lọc theo "Báo cáo sau vận hành"
    3. Xử lý cấu trúc cột
    4. Trả về kết quả cuối cùng
    """
    
    # Danh sách cột mong muốn theo đúng thứ tự
    expected_columns = [
        'Trạng thái', 'Code', 'Nhà máy', 'P.đặt', 'Q.ĐK', 'Điện lực', 'Loại',
        '00:30', '01:00', '01:30', '02:00', '02:30', '03:00', '03:30', '04:00', '04:30',
        '05:00', '05:30', '06:00', '06:30', '07:00', '07:30', '08:00', '08:30', '09:00',
        '09:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30',
        '14:00', '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00',
        '18:30', '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:00', '22:30',
        '23:00', '23:30', '00:00'
    ]
    
    df_list = []
    total_files = 0
    successful_files = 0
    failed_files = 0
    files_by_date = {}
    
    print(f"🚀 Bắt đầu đọc và xử lý dữ liệu A2/SPC từ: {root_path}")
    
    # BƯỚC 1: ĐỌC DỮ LIỆU
    for year in os.listdir(root_path):
        year_path = os.path.join(root_path, year)
        if not os.path.isdir(year_path):
            continue
        
        print(f"\n📅 Đang xử lý năm: {year}")
        
        for month in range(1, 13):
            folder_path = os.path.join(year_path, f'{month:02d}')
            
            if not os.path.exists(folder_path):
                continue
            
            print(f"  📂 Tháng {month:02d}...")
            month_files = 0
            
            for file in os.listdir(folder_path):
                if file.endswith('.xlsx'):
                    if '-A2' not in file and '-SPC' not in file:
                        continue
                        
                    total_files += 1
                    date_match = re.search(r'(\d{4}\.\d{2}\.\d{2})', file)
                    date_part = date_match.group(1) if date_match else ""
                    file_path = os.path.join(folder_path, file)
                    
                    try:
                        if '-A2' in file:
                            file_type = 'A2'
                        elif '-SPC' in file:
                            file_type = 'SPC'
                        
                        if date_part:
                            if date_part not in files_by_date:
                                files_by_date[date_part] = set()
                            files_by_date[date_part].add(file_type)
                        
                        df_full = pd.read_excel(file_path, header=None)
                        
                        # Tìm vị trí cột "Trạng thái"
                        start_row, status_col = None, None
                        found = False
                        
                        for r_idx, row in df_full.iterrows():
                            for c_idx, cell_value in enumerate(row):
                                if str(cell_value).strip() == 'Trạng thái':
                                    start_row = r_idx
                                    status_col = c_idx
                                    found = True
                                    break
                            if found:
                                break
                        
                        if start_row is not None:
                            header_row = df_full.iloc[start_row]
                            
                            # Tìm vị trí cột "00:00"
                            end_col = None
                            for c_idx, cell_value in enumerate(header_row):
                                if str(cell_value).strip() == '00:00':
                                    end_col = c_idx
                                    break
                            
                            if end_col is not None:
                                data_cols = list(range(status_col, end_col + 1))
                                actual_header = [str(header_row.iloc[i]).strip() for i in data_cols]
                                df_data = df_full.iloc[start_row + 1:, data_cols].copy()
                                
                                df_data.columns = actual_header
                                df_data.dropna(how='all', inplace=True)
                                df_data.reset_index(drop=True, inplace=True)
                                
                                if len(actual_header) >= len(expected_columns):
                                    df_data = df_data.iloc[:, :len(expected_columns)]
                                    df_data.columns = expected_columns
                                    
                                    # Thêm thông tin meta
                                    df_data.insert(0, 'Ngày', date_part)
                                    df_data.insert(1, 'Loại_File', file_type)
                                    
                                    df_list.append(df_data)
                                    successful_files += 1
                                    month_files += 1
                                    
                                    if total_files % 20 == 0:
                                        print(f"    ✅ Đã xử lý {total_files} file...")
                                else:
                                    failed_files += 1
                            else:
                                failed_files += 1
                        else:
                            failed_files += 1
                            
                    except Exception as e:
                        print(f"    ❌ Lỗi file {file}: {e}")
                        failed_files += 1
            
            if month_files > 0:
                print(f"    ✅ Hoàn thành tháng {month:02d}: {month_files} file")
    
    # Kiểm tra file thiếu
    check_missing_a2_spc_files(files_by_date)
    
    if not df_list:
        print(f'\n❌ Không đọc được dữ liệu A2/SPC nào')
        return pd.DataFrame()
    
    # Gộp tất cả dữ liệu
    df_raw = pd.concat(df_list, ignore_index=True)
    print(f'\n📊 ĐỌC DỮ LIỆU THÀNH CÔNG:')
    print(f'   - Tổng file A2/SPC kiểm tra: {total_files}')
    print(f'   - File đọc thành công: {successful_files}')
    print(f'   - File lỗi: {failed_files}')
    print(f'   - Tổng số dòng dữ liệu thô: {len(df_raw):,}')
    
    # BƯỚC 2: XỬ LÝ VÀ LỌC DỮ LIỆU
    print(f'\n🔄 ĐANG XỬ LÝ VÀ LỌC DỮ LIỆU...')
    
    # Lọc chỉ lấy "Báo cáo sau vận hành"
    df_filtered = df_raw[df_raw['Loại'].isin(['Báo cáo sau vận hành', 'Báo cáo sau vận hành tổng hợp'])]
    print(f'   - Sau khi lọc theo loại báo cáo: {len(df_filtered):,} dòng')
    
    if df_filtered.empty:
        print(f'❌ Không có dữ liệu sau khi lọc!')
        return pd.DataFrame()
    
    # Loại bỏ các cột không cần thiết
    df_processed = df_filtered.drop(columns=['Code', 'Loại', 'Trạng thái']).reset_index(drop=True)
    print(f'   - Sau khi loại bỏ cột không cần thiết: {len(df_processed.columns)} cột')
    
    # Sắp xếp lại cột: Ngày đặt trước cột 00:30
    if 'Ngày' in df_processed.columns and '00:30' in df_processed.columns:
        cols = df_processed.columns.tolist()
        cols.remove('Ngày')
        time_30_index = cols.index('00:30')
        cols.insert(time_30_index, 'Ngày')
        df_processed = df_processed[cols]
        print(f'   - Đã sắp xếp lại thứ tự cột')
    
    # BƯỚC 3: KẾT QUẢ CUỐI CÙNG
    print(f'\n🎉 HOÀN THÀNH XỬ LÝ DỮ LIỆU:')
    print(f'   - Tổng số dòng cuối cùng: {len(df_processed):,}')
    print(f'   - Số cột cuối cùng: {len(df_processed.columns)}')
    
    # Hiển thị thống kê
    if 'Loại_File' in df_processed.columns:
        print(f'\n📁 Thống kê theo loại file:')
        file_stats = df_processed['Loại_File'].value_counts()
        for file_type, count in file_stats.items():
            print(f'   - {file_type}: {count:,} dòng')
    
    if 'Ngày' in df_processed.columns:
        print(f'\n📆 Khoảng thời gian:')
        dates = pd.to_datetime(df_processed['Ngày'], format='%Y.%m.%d', errors='coerce')
        dates = dates.dropna()
        if not dates.empty:
            print(f'   - Từ: {dates.min().strftime("%Y.%m.%d")}')
            print(f'   - Đến: {dates.max().strftime("%Y.%m.%d")}')
    
    print(f'\n📋 CẤU TRÚC CỘT CUỐI CÙNG:')
    for i, col in enumerate(df_processed.columns, 1):
        print(f'   {i:2d}. {col}')
    
    return df_processed

def read_and_process_a2_spc_by_date_range(start_date_str, end_date_str, root_path=r'P:/15. Hệ thống DGMS/Daily Report/Nam/BCSVH_xlsx'):
    """
    Đọc và xử lý dữ liệu A2/SPC theo khoảng ngày - luồng thống nhất
    """
    
    try:
        start_date = datetime.strptime(start_date_str, '%Y%m%d')
        end_date = datetime.strptime(end_date_str, '%Y%m%d')
        
        if start_date > end_date:
            print("❌ Ngày bắt đầu không thể lớn hơn ngày kết thúc!")
            return pd.DataFrame()
            
    except ValueError:
        print("❌ Định dạng ngày không đúng! Vui lòng sử dụng format YYYYMMDD")
        return pd.DataFrame()
    
    expected_columns = [
        'Trạng thái', 'Code', 'Nhà máy', 'P.đặt', 'Q.ĐK', 'Điện lực', 'Loại',
        '00:30', '01:00', '01:30', '02:00', '02:30', '03:00', '03:30', '04:00', '04:30',
        '05:00', '05:30', '06:00', '06:30', '07:00', '07:30', '08:00', '08:30', '09:00',
        '09:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30',
        '14:00', '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00',
        '18:30', '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:00', '22:30',
        '23:00', '23:30', '00:00'
    ]
    
    df_list = []
    total_files = 0
    successful_files = 0
    files_by_date = {}
    
    # Tạo danh sách ngày target
    current_date = start_date
    target_dates = set()
    
    while current_date <= end_date:
        date_str = current_date.strftime('%Y.%m.%d')
        target_dates.add(date_str)
        files_by_date[date_str] = set()
        current_date += timedelta(days=1)
    
    print(f"🚀 Đọc và xử lý dữ liệu A2/SPC từ {start_date.strftime('%Y.%m.%d')} đến {end_date.strftime('%Y.%m.%d')}")
    print(f"📊 Tổng cộng {len(target_dates)} ngày cần xử lý")
    
    # BƯỚC 1: ĐỌC DỮ LIỆU THEO KHOẢNG NGÀY
    years_in_range = set()
    for date_str in target_dates:
        year = date_str.split('.')[0]
        years_in_range.add(year)
    
    for year in sorted(years_in_range):
        year_path = os.path.join(root_path, year)
        
        if not os.path.exists(year_path):
            continue
        
        print(f"\n📅 Đang xử lý năm: {year}")
        
        for month in range(1, 13):
            folder_path = os.path.join(year_path, f'{month:02d}')
            
            if not os.path.exists(folder_path):
                continue
            
            for file in os.listdir(folder_path):
                if file.endswith('.xlsx'):
                    if '-A2' not in file and '-SPC' not in file:
                        continue
                        
                    date_match = re.search(r'(\d{4}\.\d{2}\.\d{2})', file)
                    date_part = date_match.group(1) if date_match else ""
                    
                    if date_part not in target_dates:
                        continue
                    
                    total_files += 1
                    file_path = os.path.join(folder_path, file)
                    
                    try:
                        if '-A2' in file:
                            file_type = 'A2'
                        elif '-SPC' in file:
                            file_type = 'SPC'
                        
                        files_by_date[date_part].add(file_type)
                        
                        df_full = pd.read_excel(file_path, header=None)
                        
                        start_row, status_col = None, None
                        found = False
                        
                        for r_idx, row in df_full.iterrows():
                            for c_idx, cell_value in enumerate(row):
                                if str(cell_value).strip() == 'Trạng thái':
                                    start_row = r_idx
                                    status_col = c_idx
                                    found = True
                                    break
                            if found:
                                break
                        
                        if start_row is not None:
                            header_row = df_full.iloc[start_row]
                            
                            end_col = None
                            for c_idx, cell_value in enumerate(header_row):
                                if str(cell_value).strip() == '00:00':
                                    end_col = c_idx
                                    break
                            
                            if end_col is not None:
                                data_cols = list(range(status_col, end_col + 1))
                                actual_header = [str(header_row.iloc[i]).strip() for i in data_cols]
                                df_data = df_full.iloc[start_row + 1:, data_cols].copy()
                                
                                df_data.columns = actual_header
                                df_data.dropna(how='all', inplace=True)
                                df_data.reset_index(drop=True, inplace=True)
                                
                                if len(actual_header) >= len(expected_columns):
                                    df_data = df_data.iloc[:, :len(expected_columns)]
                                    df_data.columns = expected_columns
                                    
                                    df_data.insert(0, 'Ngày', date_part)
                                    df_data.insert(1, 'Loại_File', file_type)
                                    
                                    df_list.append(df_data)
                                    successful_files += 1
                                    
                                    if total_files % 10 == 0:
                                        print(f"    ✅ Đã xử lý {total_files} file...")
                        
                    except Exception as e:
                        print(f"    ❌ Lỗi file {file}: {e}")
    
    # Kiểm tra file thiếu
    check_missing_a2_spc_files(files_by_date, target_dates)
    
    if not df_list:
        print(f'\n❌ Không đọc được dữ liệu A2/SPC nào trong khoảng ngày đã chọn')
        return pd.DataFrame()
    
    # Gộp dữ liệu thô
    df_raw = pd.concat(df_list, ignore_index=True)
    print(f'\n📊 ĐỌC DỮ LIỆU THÀNH CÔNG:')
    print(f'   - Tổng file A2/SPC kiểm tra: {total_files}')
    print(f'   - File đọc thành công: {successful_files}')
    print(f'   - Tổng số dòng dữ liệu thô: {len(df_raw):,}')
    
    # BƯỚC 2: XỬ LÝ VÀ LỌC DỮ LIỆU
    print(f'\n🔄 ĐANG XỬ LÝ VÀ LỌC DỮ LIỆU...')
    
    df_filtered = df_raw[df_raw['Loại'].isin(['Báo cáo sau vận hành', 'Báo cáo sau vận hành tổng hợp'])]
    print(f'   - Sau khi lọc theo loại báo cáo: {len(df_filtered):,} dòng')
    
    if df_filtered.empty:
        print(f'❌ Không có dữ liệu sau khi lọc!')
        return pd.DataFrame()
    
    df_processed = df_filtered.drop(columns=['Code', 'Loại', 'Trạng thái']).reset_index(drop=True)
    print(f'   - Sau khi loại bỏ cột không cần thiết: {len(df_processed.columns)} cột')
    
    # Sắp xếp lại cột
    if 'Ngày' in df_processed.columns and '00:30' in df_processed.columns:
        cols = df_processed.columns.tolist()
        cols.remove('Ngày')
        time_30_index = cols.index('00:30')
        cols.insert(time_30_index, 'Ngày')
        df_processed = df_processed[cols]
        print(f'   - Đã sắp xếp lại thứ tự cột')
    
    # Kết quả cuối cùng
    print(f'\n🎉 HOÀN THÀNH XỬ LÝ DỮ LIỆU:')
    print(f'   - Tổng số dòng cuối cùng: {len(df_processed):,}')
    print(f'   - Số cột cuối cùng: {len(df_processed.columns)}')
    
    return df_processed

def check_missing_a2_spc_files(files_by_date, target_dates=None):
    """Kiểm tra file A2 và SPC thiếu cho từng ngày"""
    
    required_files = {'A2', 'SPC'}
    missing_reports = []
    complete_days = 0
    
    dates_to_check = target_dates if target_dates else files_by_date.keys()
    
    for date in sorted(dates_to_check):
        available_files = files_by_date.get(date, set())
        missing_files = required_files - available_files
        
        if not missing_files:
            complete_days += 1
        elif len(missing_files) == 2:
            missing_reports.append(f"   📅 {date}: ❌ Thiếu cả A2 và SPC")
        else:
            missing_file = list(missing_files)[0]
            missing_reports.append(f"   📅 {date}: ⚠️ Thiếu file {missing_file}")
    
    print(f'\n📋 BÁO CÁO FILE A2/SPC THIẾU:')
    total_days = len(dates_to_check)
    missing_days = total_days - complete_days
    
    print(f'   - Tổng số ngày kiểm tra: {total_days}')
    print(f'   - Ngày đầy đủ file (A2 + SPC): {complete_days}')
    print(f'   - Ngày thiếu file: {missing_days}')
    
    if missing_reports:
        print(f'\n⚠️ CHI TIẾT NGÀY THIẾU FILE:')
        for report in missing_reports:
            print(report)
    else:
        print(f'\n✅ Tất cả ngày đều có đầy đủ file A2 và SPC!')

# ===== SỬ DỤNG LUỒNG THỐNG NHẤT =====

print("🔥 LUỒNG THỐNG NHẤT: Đọc và xử lý dữ liệu A2/SPC!")
user_confirm = input("Bạn có muốn đọc tất cả dữ liệu A2/SPC? (y/n): ")

if user_confirm.lower() == 'y':
    # Đọc và xử lý tất cả dữ liệu
    df_nam_final = read_and_process_a2_spc_data()
    
    if not df_nam_final.empty:
        print(f"\n✅ Dữ liệu đã sẵn sàng sử dụng: df_nam_final")
        print(f"Shape: {df_nam_final.shape}")

else:
    print("\n📅 Nhập khoảng ngày cần đọc và xử lý dữ liệu A2/SPC:")
    
    # Nhập ngày bắt đầu
    while True:
        start_date_input = input("Ngày bắt đầu (YYYYMMDD): ").strip()
        try:
            datetime.strptime(start_date_input, '%Y%m%d')
            break
        except ValueError:
            print("❌ Định dạng không đúng! Vui lòng nhập theo format YYYYMMDD")
    
    # Nhập ngày kết thúc
    while True:
        end_date_input = input("Ngày kết thúc (YYYYMMDD): ").strip()
        try:
            end_date = datetime.strptime(end_date_input, '%Y%m%d')
            start_date = datetime.strptime(start_date_input, '%Y%m%d')
            
            if start_date <= end_date:
                break
            else:
                print("❌ Ngày kết thúc phải lớn hơn hoặc bằng ngày bắt đầu!")
        except ValueError:
            print("❌ Định dạng không đúng! Vui lòng nhập theo format YYYYMMDD")
    
    # Đọc và xử lý dữ liệu theo khoảng ngày
    df_nam_final = read_and_process_a2_spc_by_date_range(start_date_input, end_date_input)
    
    if not df_nam_final.empty:
        print(f"\n✅ Dữ liệu đã sẵn sàng sử dụng: df_nam_final")
        print(f"Shape: {df_nam_final.shape}")

print("\n🎯 Hoàn thành luồng thống nhất!")

🔥 LUỒNG THỐNG NHẤT: Đọc và xử lý dữ liệu A2/SPC!

📅 Nhập khoảng ngày cần đọc và xử lý dữ liệu A2/SPC:


In [2]:
import os

# Tạo thư mục nếu chưa tồn tại
output_dir = r"D:\OneDrive\0. Code\TDN\data\interim\Nam"
os.makedirs(output_dir, exist_ok=True)

# Lưu dữ liệu ra file parquet
if 'df_nam_final' in locals() and not df_nam_final.empty:
    # Xử lý cột "Nhà máy" để tránh lỗi mixed types
    print("🔧 Đang xử lý kiểu dữ liệu...")
    
    # Kiểm tra dữ liệu trong cột "Nhà máy"
    print(f"Kiểu dữ liệu của cột 'Nhà máy': {df_nam_final['Nhà máy'].dtype}")
    print(f"Các giá trị unique trong cột 'Nhà máy': {df_nam_final['Nhà máy'].unique()[:10]}")
    
    # Tạo bản copy để xử lý
    df_save = df_nam_final.copy()
    
    # Chuyển đổi cột "Nhà máy" về kiểu string thống nhất
    df_save['Nhà máy'] = df_save['Nhà máy'].astype(str)
    
    # Xử lý các cột khác có thể có vấn đề tương tự
    object_columns = df_save.select_dtypes(include=['object']).columns
    for col in object_columns:
        try:
            df_save[col] = df_save[col].astype(str)
        except:
            print(f"⚠️ Không thể chuyển đổi cột {col}")
    
    # Lưu file
    output_file = os.path.join(output_dir, "df_nam_final.parquet")
    df_save.to_parquet(output_file, index=False)
    print(f"✅ Đã lưu dữ liệu ra file: {output_file}")
    print(f"📊 Kích thước file: {os.path.getsize(output_file) / (1024*1024):.2f} MB")
    print(f"📋 Shape: {df_save.shape}")
else:
    print("❌ Không có dữ liệu để lưu!")

🔧 Đang xử lý kiểu dữ liệu...
Kiểu dữ liệu của cột 'Nhà máy': object
Các giá trị unique trong cột 'Nhà máy': ['Đăk Gkun' 'Đăk Gkun 2' 'Krong No 2' 'Bảo Lộc' 'YanTannSien' 'Krong No 3'
 'Đồng Nai 1' 'Đa Dâng 3' 'Sông Cái' 'Hạ Sông Pha 1']
✅ Đã lưu dữ liệu ra file: D:\Khuê\Khue\TDN\data\interim\Nam\df_nam_final.parquet
📊 Kích thước file: 1.03 MB
📋 Shape: (14215, 54)


In [3]:
# Đọc dữ liệu tham chiếu
df_thamchieu = pd.read_excel(r"D:\Khuê\Khue\TDN\data\raw\DS_TTT_TDN.xlsx")
print(f"✅ Đọc thành công df_thamchieu - Shape: {df_thamchieu.shape}")

✅ Đọc thành công df_thamchieu - Shape: (537, 20)


In [4]:
df_thamchieu

,ID_NM,DIEU_DO,TCTDL,DIEN_LUC,TENNHAMAY,TEN_NM,LOAI_HINH,QUYEN_DIEUKHIEN,CS_DAT,A_TB_NAM,U_DAUNOI,MIEN,TINH,HUYEN,KINHDO,VIDO,BACTHANGSONG,TD_THAMCHIEU,HIEU_LUC,DVPD
0,10997,A1,NPC,PC_BACKAN,Nặm Cắt,NAM_CAT,1,PC_BACKAN,3.2,0.000,35,Bắc,Bắc Kạn,Bạch Thông,105.789306,22.192518,SONG_HONG,TUYEN_QUANG,1,NaN
1,10998,A1,NPC,PC_BACKAN,Pác Cáp,PAC_CAP,1,PC_BACKAN,6.0,0.000,0,NaN,NaN,NaN,106.127303,22.210818,SONG_HONG,TUYEN_QUANG,1,NaN
2,10995,A1,NPC,PC_BACKAN,Tà Làng,TA_LANG,1,PC_BACKAN,4.5,0.000,0,NaN,NaN,NaN,105.721185,22.303931,SONG_HONG,TUYEN_QUANG,1,NaN
3,11193,A1,NPC,PC_BACKAN,THÁC GIỀNG 1,THAC_GIENG 1,1,PC_BACKAN,5.5,0.000,0,NaN,NaN,NaN,105.893735,22.095786,SONG_HONG,TUYEN_QUANG,1,NaN
4,11191,A1,NPC,PC_BACKAN,Thượng Ân,THUONG_AN,1,PC_BACKAN,2.4,0.000,0,NaN,NaN,NaN,106.003945,22.511839,SONG_HONG,TUYEN_QUANG,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,11054,A1,NPC,PC_YENBAI,Thào Sa Chải,THAO_SA_CHAI,1,PC_YENBAI,6.5,22.684,35,Bắc,Yên Bái,Mù Cang Chải,104.268669,21.838929,SONG_HONG,BAN_CHAT,0,NaN
533,10830,A1,NPC,PC_YENBAI,Trạm Tấu,TRAM_TAU,1,A1,30.0,0.000,0,NaN,NaN,NaN,104.442036,21.498660,SONG_HONG,BAN_CHAT,1,NaN
534,10840,A1,NPC,PC_YENBAI,Vực Tuần,VUC_TUAN,1,PC_YENBAI,5.0,0.000,0,NaN,NaN,NaN,104.569095,21.416027,SONG_HONG,BAN_CHAT,1,NaN
535,10837,A1,NPC,PC_YENBAI,Nậm Tục,NAM_TUC,1,PC_YENBAI,3.0,0.000,0,NaN,NaN,NaN,104.447998,21.608201,SONG_HONG,BAN_CHAT,1,NaN


In [5]:
df_nam_final

,Loại_File,Nhà máy,P.đặt,Q.ĐK,Điện lực,Ngày,00:30,01:00,01:30,02:00,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,00:00
0,A2,Đăk Gkun,18,A2,PC_BINHPHUOC,2024.08.25,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A2,Đăk Gkun 2,10,A2,PC_BINHPHUOC,2024.08.25,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A2,Krong No 2,29,A2,PC_LAMDONG,2024.08.25,14.5,14.5,14.5,14.5,...,29,29,29,29,29,29,29,29,29,29
3,A2,Bảo Lộc,25,A2,PC_LAMDONG,2024.08.25,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A2,YanTannSien,19.5,A2,PC_LAMDONG,2024.08.25,19.5,19.5,19.5,19.5,...,18,18,18,18,18,18,18,18,18,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14210,SPC,Thượng Sông Ông,7.5,PC_NINHTHUAN,PC_NINHTHUAN,2025.05.31,6.48,6.88,5.83,4.95,...,6.32,6.3,5.15,5.06,5.14,5.21,5.25,5.23,5.23,5.24
14211,SPC,Thượng Sông Ông 1,6.6,PC_NINHTHUAN,PC_NINHTHUAN,2025.05.31,3.54,3.36,4.26,4.88,...,6.58,6.56,4.64,4.12,4.72,5.06,5.06,5.14,5.26,5.26
14212,SPC,Mia duong Soc Trang,12,PC_SOCTRANG,PC_SOCTRANG,2025.05.31,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14213,SPC,CS 2,1.5,PC_TAYNINH,PC_TAYNINH,2025.05.31,0.36,0.36,0.36,0.36,...,0.36,0.36,0.36,0.36,0.36,0.36,0.36,0.36,0.36,0.36


In [6]:
# Merge dữ liệu tham chiếu vào df_nam_final
print("🔄 Đang merge dữ liệu tham chiếu...")

# Kiểm tra các cột key trước khi merge
print(f"📊 Số lượng nhà máy unique trong df_nam_final: {df_nam_final['Nhà máy'].nunique()}")
print(f"📊 Số lượng nhà máy unique trong df_thamchieu: {df_thamchieu['TENNHAMAY'].nunique()}")

# Thực hiện merge với left join để giữ tất cả dữ liệu từ df_nam_final
df_merged = df_nam_final.merge(
    df_thamchieu[['TENNHAMAY', 'TD_THAMCHIEU']], 
    left_on='Nhà máy', 
    right_on='TENNHAMAY', 
    how='left'
)

print(f"✅ Merge hoàn thành - Shape: {df_merged.shape}")
print(f"📋 Số dòng trước merge: {df_nam_final.shape[0]}")
print(f"📋 Số dòng sau merge: {df_merged.shape[0]}")

# Kiểm tra số lượng giá trị null trong cột TD_THAMCHIEU
null_count = df_merged['TD_THAMCHIEU'].isnull().sum()
print(f"⚠️ Số dòng không có thông tin tham chiếu: {null_count}")

# Hiển thị một số dòng mẫu
print("\n📝 Dữ liệu sau merge:")
print(df_merged[['Nhà máy', 'TENNHAMAY', 'TD_THAMCHIEU']].head())

# Cập nhật df_nam_final
df_nam_final = df_merged.drop(columns=['TENNHAMAY'])
print(f"🔄 Đã cập nhật df_nam_final - Shape: {df_nam_final.shape}")

🔄 Đang merge dữ liệu tham chiếu...
📊 Số lượng nhà máy unique trong df_nam_final: 56
📊 Số lượng nhà máy unique trong df_thamchieu: 534
✅ Merge hoàn thành - Shape: (14215, 56)
📋 Số dòng trước merge: 14215
📋 Số dòng sau merge: 14215
⚠️ Số dòng không có thông tin tham chiếu: 2928

📝 Dữ liệu sau merge:
       Nhà máy    TENNHAMAY TD_THAMCHIEU
0     Đăk Gkun     Đăk Gkun     DAK_RTIH
1   Đăk Gkun 2   Đăk Gkun 2     DAK_RTIH
2   Krong No 2          NaN          NaN
3      Bảo Lộc      Bảo Lộc     DAK_RTIH
4  YanTannSien  YanTannSien     SREPOK_3
🔄 Đã cập nhật df_nam_final - Shape: (14215, 55)


In [8]:
# Xuất dữ liệu ra file Excel vào thư mục Nam
print("📤 Đang xuất dữ liệu ra file Excel...")

# Tạo tên file với date
from datetime import datetime
date_str = datetime.now().strftime("%Y%m%d")
output_file = f"D:/Khuê/Khue/TDN/data/interim/Nam/DGMS_data_{date_str}.xlsx"

# Xuất df_nam_final ra Excel
df_nam_final.to_excel(output_file, index=False)

print(f"✅ Xuất file thành công: {output_file}")
print(f"📊 Dữ liệu đã được lưu với {df_nam_final.shape[0]} dòng và {df_nam_final.shape[1]} cột")

📤 Đang xuất dữ liệu ra file Excel...
✅ Xuất file thành công: D:/Khuê/Khue/TDN/data/interim/Nam/DGMS_data_20250617.xlsx
📊 Dữ liệu đã được lưu với 14215 dòng và 55 cột


In [1]:
import pandas as pd
import warnings
import os
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import calendar
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

# Đọc file Sanluong
sanluong_path_nam = r"D:\OneDrive\0. Code\TDN\data\interim\Nam\DGMS_data_20250617.xlsx"
df_sanluong_nam = pd.read_excel(sanluong_path_nam, engine='openpyxl')
df_sanluong_nam

,Loại_File,Nhà máy,P.đặt,Q.ĐK,Điện lực,TD_THAMCHIEU,Ngày,00:30,01:00,01:30,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,00:00
0,A2,Đăk Gkun,18.0,A2,PC_BINHPHUOC,DAK_RTIH,2024.08.25,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,A2,Đăk Gkun 2,10.0,A2,PC_BINHPHUOC,DAK_RTIH,2024.08.25,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,A2,Krong No 2,29.0,A2,PC_LAMDONG,BUON_TUA_SRAH,2024.08.25,14.50,14.50,14.50,...,29.00,29.00,29.00,29.00,29.00,29.00,29.00,29.00,29.00,29.00
3,A2,Bảo Lộc,25.0,A2,PC_LAMDONG,DAK_RTIH,2024.08.25,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,A2,YanTannSien,19.5,A2,PC_LAMDONG,BUON_TUA_SRAH,2024.08.25,19.50,19.50,19.50,...,18.00,18.00,18.00,18.00,18.00,18.00,18.00,18.00,18.00,18.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13694,SPC,Sông Ông,8.1,PC_NINHTHUAN,PC_NINHTHUAN,DAK_RTIH,2025.05.31,8.00,8.00,8.07,...,8.32,8.33,8.32,7.36,7.07,7.08,7.53,7.03,7.31,7.31
13695,SPC,Thượng Sông Ông,7.5,PC_NINHTHUAN,PC_NINHTHUAN,DAK_RTIH,2025.05.31,6.48,6.88,5.83,...,6.32,6.30,5.15,5.06,5.14,5.21,5.25,5.23,5.23,5.24
13696,SPC,Thượng Sông Ông 1,6.6,PC_NINHTHUAN,PC_NINHTHUAN,DAK_RTIH,2025.05.31,3.54,3.36,4.26,...,6.58,6.56,4.64,4.12,4.72,5.06,5.06,5.14,5.26,5.26
13697,SPC,CS 2,1.5,PC_TAYNINH,PC_TAYNINH,DAK_RTIH,2025.05.31,0.36,0.36,0.36,...,0.36,0.36,0.36,0.36,0.36,0.36,0.36,0.36,0.36,0.36


In [2]:
# Kiểm tra lại các dòng còn thiếu giá trị trong cột "P.Đặt"
missing_pdat = df_sanluong_nam[df_sanluong_nam['P.đặt'].isnull()]
# Kiểm tra các dòng còn thiếu giá trị trong cột "P.Đặt"
missing_pdat = df_sanluong_nam[df_sanluong_nam['P.đặt'].isnull()]
if not missing_pdat.empty:
    print(f"Còn {len(missing_pdat)} dòng thiếu giá trị trong cột 'P.đặt':")
    print(missing_pdat[['Nhà máy', 'TD_THAMCHIEU', 'P.đặt']].drop_duplicates())
else:
    print("Không còn dòng nào thiếu giá trị trong cột 'P.đặt'")

Không còn dòng nào thiếu giá trị trong cột 'P.đặt'


In [3]:
df_sanluong_nam['SL'] = df_sanluong_nam.loc[:, '00:30':'00:00'].sum(axis=1) / 2
df_dgms_sanluong_nam = df_sanluong_nam[['TD_THAMCHIEU', 'Ngày', 'SL', 'P.đặt']].groupby(['TD_THAMCHIEU', 'Ngày'], as_index=False).sum()
df_dgms_sanluong_nam

,TD_THAMCHIEU,Ngày,SL,P.đặt
0,BUON_TUA_SRAH,2024.08.25,826.000,66.5
1,BUON_TUA_SRAH,2024.08.26,780.000,66.5
2,BUON_TUA_SRAH,2024.08.27,1098.750,66.5
3,BUON_TUA_SRAH,2024.08.28,217.500,66.5
4,BUON_TUA_SRAH,2024.08.29,594.000,66.5
...,...,...,...,...
555,DAK_RTIH,2025.05.27,4142.045,370.6
556,DAK_RTIH,2025.05.28,5242.720,370.6
557,DAK_RTIH,2025.05.29,4343.935,370.6
558,DAK_RTIH,2025.05.30,4341.440,370.6


In [4]:
# Chuyển cột 'SL' và 'P.đặt' sang kiểu số
df_dgms_sanluong_nam['SL'] = pd.to_numeric(df_dgms_sanluong_nam['SL'], errors='coerce')
df_dgms_sanluong_nam['P.đặt'] = pd.to_numeric(df_dgms_sanluong_nam['P.đặt'], errors='coerce')
# Chuyển giá trị cột 'SL' từ MWh sang triệu kWh
df_dgms_sanluong_nam['SL'] = df_dgms_sanluong_nam['SL'] / 1000  # Chia cho 1000 để chuyển từ MWh sang triệu kWh

In [5]:
unique_td_thamchieu = df_dgms_sanluong_nam['TD_THAMCHIEU'].unique()
print(f"Số lượng giá trị unique trong cột TD_THAMCHIEU: {len(unique_td_thamchieu)}")
print("\nCác giá trị unique trong cột TD_THAMCHIEU:")
for value in sorted(unique_td_thamchieu):
    print(f"- {value}")

Số lượng giá trị unique trong cột TD_THAMCHIEU: 2

Các giá trị unique trong cột TD_THAMCHIEU:
- BUON_TUA_SRAH
- DAK_RTIH


In [6]:
df_Qve_nam = pd.read_excel(r"D:\OneDrive\0. Code\TDN\data\interim\Nam\Q_ve_nam.xlsx", engine='openpyxl')
df_Qve_nam['NGAY_VANHANH'] = pd.to_datetime(df_Qve_nam['NGAY_VANHANH']).dt.strftime('%Y.%m.%d')
df_Qve_nam

,TENTAT_NM,NGAY_VANHANH,SANLUONG_NGAY,QVE,QMAY,QXA,HTL
0,BUON_TUA_SRAH,2021.01.01,658455.0,79.00,54.46,0.00,487068.00
1,BUON_TUA_SRAH,2021.01.02,986733.0,73.63,81.62,0.00,487049.00
2,BUON_TUA_SRAH,2021.01.03,1022145.0,69.63,84.71,0.00,487014.00
3,BUON_TUA_SRAH,2021.01.04,1559998.0,70.21,129.42,0.00,486876.00
4,BUON_TUA_SRAH,2021.01.05,874818.0,76.38,72.21,0.00,486886.00
...,...,...,...,...,...,...,...
4882,DAK_RTIH,2025.06.14,2281.5,48.98,48.63,1.20,413.90
4883,DAK_RTIH,2025.06.15,620.7,21.62,19.83,1.20,414.35
4884,DAK_RTIH,2025.06.16,2684.1,54.40,54.42,1.20,413.42
4885,DAK_RTIH,2025.06.17,2486.8,52.55,51.07,1.20,414.55


In [7]:
# chuyển giá trị cột 'Ngày' trong df_dgms_sanluong và 'NGAY_VANHANH' trong df_Qve về định dạng datetime
df_dgms_sanluong_nam['Ngày'] = pd.to_datetime(df_dgms_sanluong_nam['Ngày'])
df_Qve_nam['NGAY_VANHANH'] = pd.to_datetime(df_Qve_nam['NGAY_VANHANH'])

In [8]:
# Chuẩn hóa định dạng cột TD_THAMCHIEU và TENTAT_NM để đảm bảo merge chính xác
df_dgms_sanluong_nam['TD_THAMCHIEU'] = df_dgms_sanluong_nam['TD_THAMCHIEU'].astype(str).str.strip()
df_Qve_nam['TENTAT_NM'] = df_Qve_nam['TENTAT_NM'].astype(str).str.strip()

# Kiểm tra các giá trị unique để đảm bảo mapping đúng
print("Các giá trị unique trong TD_THAMCHIEU:", df_dgms_sanluong_nam['TD_THAMCHIEU'].unique())
print("Các giá trị unique trong TENTAT_NM:", df_Qve_nam['TENTAT_NM'].unique())

Các giá trị unique trong TD_THAMCHIEU: ['BUON_TUA_SRAH' 'DAK_RTIH']
Các giá trị unique trong TENTAT_NM: ['BUON_TUA_SRAH' 'DAK_RTIH']


In [9]:
# Merge dữ liệu QVE từ df_Qve vào df_dgms_sanluong
df_dgms_sanluong_nam_merge = df_dgms_sanluong_nam.merge(
    df_Qve_nam[['TENTAT_NM', 'NGAY_VANHANH', 'QVE']], 
    left_on=['TD_THAMCHIEU', 'Ngày'], 
    right_on=['TENTAT_NM', 'NGAY_VANHANH'], 
    how='left'
)

# Xóa các cột trung gian không cần thiết
df_dgms_sanluong_nam_merge = df_dgms_sanluong_nam_merge.drop(['TENTAT_NM', 'NGAY_VANHANH'], axis=1)

# Hiển thị kết quả
df_dgms_sanluong_nam_merge

,TD_THAMCHIEU,Ngày,SL,P.đặt,QVE
0,BUON_TUA_SRAH,2024-08-25,0.826000,66.5,133.08
1,BUON_TUA_SRAH,2024-08-26,0.780000,66.5,215.54
2,BUON_TUA_SRAH,2024-08-27,1.098750,66.5,168.08
3,BUON_TUA_SRAH,2024-08-28,0.217500,66.5,169.96
4,BUON_TUA_SRAH,2024-08-29,0.594000,66.5,136.42
...,...,...,...,...,...
835,DAK_RTIH,2025-05-29,4.343935,370.6,14.66
836,DAK_RTIH,2025-05-30,4.341440,370.6,18.33
837,DAK_RTIH,2025-05-30,4.341440,370.6,14.85
838,DAK_RTIH,2025-05-31,4.300665,370.6,25.79


In [10]:
# export df_dgms_sanluong_merge
df_dgms_sanluong_nam_merge.to_excel(r"D:\OneDrive\0. Code\TDN\data\interim\Nam\DGMS_BCSVH_sanluong_nam.xlsx", index=False)

### 3. Miền Trung

In [2]:
import pandas as pd
import os
import re
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

def read_and_process_a3_cpc_data(root_path=r'P:/15. Hệ thống DGMS/Daily Report/Trung/BCSVH_xlsx'):
    """
    Đọc và xử lý dữ liệu A3/CPC thành một luồng thống nhất:
    1. Đọc dữ liệu từ file A3 và CPC
    2. Lọc theo "Báo cáo sau vận hành"
    3. Xử lý cấu trúc cột
    4. Trả về kết quả cuối cùng
    """
    
    # Danh sách cột mong muốn theo đúng thứ tự
    expected_columns = [
        'Trạng thái', 'Code', 'Nhà máy', 'P.đặt', 'Q.ĐK', 'Điện lực', 'Loại',
        '00:30', '01:00', '01:30', '02:00', '02:30', '03:00', '03:30', '04:00', '04:30',
        '05:00', '05:30', '06:00', '06:30', '07:00', '07:30', '08:00', '08:30', '09:00',
        '09:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30',
        '14:00', '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00',
        '18:30', '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:00', '22:30',
        '23:00', '23:30', '00:00'
    ]
    
    df_list = []
    total_files = 0
    successful_files = 0
    failed_files = 0
    files_by_date = {}
    
    print(f"🚀 Bắt đầu đọc và xử lý dữ liệu A3/CPC từ: {root_path}")
    
    # BƯỚC 1: ĐỌC DỮ LIỆU
    for year in os.listdir(root_path):
        year_path = os.path.join(root_path, year)
        if not os.path.isdir(year_path):
            continue
        
        print(f"\n📅 Đang xử lý năm: {year}")
        
        for month in range(1, 13):
            folder_path = os.path.join(year_path, f'{month:02d}')
            
            if not os.path.exists(folder_path):
                continue
            
            print(f"  📂 Tháng {month:02d}...")
            month_files = 0
            
            for file in os.listdir(folder_path):
                if file.endswith('.xlsx'):
                    if '-A3' not in file and '-CPC' not in file:
                        continue
                        
                    total_files += 1
                    date_match = re.search(r'(\d{4}\.\d{2}\.\d{2})', file)
                    date_part = date_match.group(1) if date_match else ""
                    file_path = os.path.join(folder_path, file)
                    
                    try:
                        if '-A3' in file:
                            file_type = 'A3'
                        elif '-CPC' in file:
                            file_type = 'CPC'
                        
                        if date_part:
                            if date_part not in files_by_date:
                                files_by_date[date_part] = set()
                            files_by_date[date_part].add(file_type)
                        
                        df_full = pd.read_excel(file_path, header=None)
                        
                        # Tìm vị trí cột "Trạng thái"
                        start_row, status_col = None, None
                        found = False
                        
                        for r_idx, row in df_full.iterrows():
                            for c_idx, cell_value in enumerate(row):
                                if str(cell_value).strip() == 'Trạng thái':
                                    start_row = r_idx
                                    status_col = c_idx
                                    found = True
                                    break
                            if found:
                                break
                        
                        if start_row is not None:
                            header_row = df_full.iloc[start_row]
                            
                            # Tìm vị trí cột "00:00"
                            end_col = None
                            for c_idx, cell_value in enumerate(header_row):
                                if str(cell_value).strip() == '00:00':
                                    end_col = c_idx
                                    break
                            
                            if end_col is not None:
                                data_cols = list(range(status_col, end_col + 1))
                                actual_header = [str(header_row.iloc[i]).strip() for i in data_cols]
                                df_data = df_full.iloc[start_row + 1:, data_cols].copy()
                                
                                df_data.columns = actual_header
                                df_data.dropna(how='all', inplace=True)
                                df_data.reset_index(drop=True, inplace=True)
                                
                                if len(actual_header) >= len(expected_columns):
                                    df_data = df_data.iloc[:, :len(expected_columns)]
                                    df_data.columns = expected_columns
                                    
                                    # Thêm thông tin meta
                                    df_data.insert(0, 'Ngày', date_part)
                                    df_data.insert(1, 'Loại_File', file_type)
                                    
                                    df_list.append(df_data)
                                    successful_files += 1
                                    month_files += 1
                                    
                                    if total_files % 20 == 0:
                                        print(f"    ✅ Đã xử lý {total_files} file...")
                                else:
                                    failed_files += 1
                            else:
                                failed_files += 1
                        else:
                            failed_files += 1
                            
                    except Exception as e:
                        print(f"    ❌ Lỗi file {file}: {e}")
                        failed_files += 1
            
            if month_files > 0:
                print(f"    ✅ Hoàn thành tháng {month:02d}: {month_files} file")
    
    # Kiểm tra file thiếu
    check_missing_a3_cpc_files(files_by_date)
    
    if not df_list:
        print(f'\n❌ Không đọc được dữ liệu A3/CPC nào')
        return pd.DataFrame()
    
    # Gộp tất cả dữ liệu
    df_raw = pd.concat(df_list, ignore_index=True)
    print(f'\n📊 ĐỌC DỮ LIỆU THÀNH CÔNG:')
    print(f'   - Tổng file A3/CPC kiểm tra: {total_files}')
    print(f'   - File đọc thành công: {successful_files}')
    print(f'   - File lỗi: {failed_files}')
    print(f'   - Tổng số dòng dữ liệu thô: {len(df_raw):,}')
    
    # BƯỚC 2: XỬ LÝ VÀ LỌC DỮ LIỆU
    print(f'\n🔄 ĐANG XỬ LÝ VÀ LỌC DỮ LIỆU...')
    
    # Lọc chỉ lấy "Báo cáo sau vận hành"
    df_filtered = df_raw[df_raw['Loại'].isin(['Báo cáo sau vận hành', 'Báo cáo sau vận hành tổng hợp'])]
    print(f'   - Sau khi lọc theo loại báo cáo: {len(df_filtered):,} dòng')
    
    if df_filtered.empty:
        print(f'❌ Không có dữ liệu sau khi lọc!')
        return pd.DataFrame()
    
    # Loại bỏ các cột không cần thiết
    df_processed = df_filtered.drop(columns=['Code', 'Loại', 'Trạng thái']).reset_index(drop=True)
    print(f'   - Sau khi loại bỏ cột không cần thiết: {len(df_processed.columns)} cột')
    
    # Sắp xếp lại cột: Ngày đặt trước cột 00:30
    if 'Ngày' in df_processed.columns and '00:30' in df_processed.columns:
        cols = df_processed.columns.tolist()
        cols.remove('Ngày')
        time_30_index = cols.index('00:30')
        cols.insert(time_30_index, 'Ngày')
        df_processed = df_processed[cols]
        print(f'   - Đã sắp xếp lại thứ tự cột')
    
    # BƯỚC 3: KẾT QUẢ CUỐI CÙNG
    print(f'\n🎉 HOÀN THÀNH XỬ LÝ DỮ LIỆU:')
    print(f'   - Tổng số dòng cuối cùng: {len(df_processed):,}')
    print(f'   - Số cột cuối cùng: {len(df_processed.columns)}')
    
    # Hiển thị thống kê
    if 'Loại_File' in df_processed.columns:
        print(f'\n📁 Thống kê theo loại file:')
        file_stats = df_processed['Loại_File'].value_counts()
        for file_type, count in file_stats.items():
            print(f'   - {file_type}: {count:,} dòng')
    
    if 'Ngày' in df_processed.columns:
        print(f'\n📆 Khoảng thời gian:')
        dates = pd.to_datetime(df_processed['Ngày'], format='%Y.%m.%d', errors='coerce')
        dates = dates.dropna()
        if not dates.empty:
            print(f'   - Từ: {dates.min().strftime("%Y.%m.%d")}')
            print(f'   - Đến: {dates.max().strftime("%Y.%m.%d")}')
    
    print(f'\n📋 CẤU TRÚC CỘT CUỐI CÙNG:')
    for i, col in enumerate(df_processed.columns, 1):
        print(f'   {i:2d}. {col}')
    
    return df_processed

def read_and_process_a3_cpc_by_date_range(start_date_str, end_date_str, root_path=r'P:/15. Hệ thống DGMS/Daily Report/Trung/BCSVH_xlsx'):
    """
    Đọc và xử lý dữ liệu A3/CPC theo khoảng ngày - luồng thống nhất
    """
    
    try:
        start_date = datetime.strptime(start_date_str, '%Y%m%d')
        end_date = datetime.strptime(end_date_str, '%Y%m%d')
        
        if start_date > end_date:
            print("❌ Ngày bắt đầu không thể lớn hơn ngày kết thúc!")
            return pd.DataFrame()
            
    except ValueError:
        print("❌ Định dạng ngày không đúng! Vui lòng sử dụng format YYYYMMDD")
        return pd.DataFrame()
    
    expected_columns = [
        'Trạng thái', 'Code', 'Nhà máy', 'P.đặt', 'Q.ĐK', 'Điện lực', 'Loại',
        '00:30', '01:00', '01:30', '02:00', '02:30', '03:00', '03:30', '04:00', '04:30',
        '05:00', '05:30', '06:00', '06:30', '07:00', '07:30', '08:00', '08:30', '09:00',
        '09:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30',
        '14:00', '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00',
        '18:30', '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:00', '22:30',
        '23:00', '23:30', '00:00'
    ]
    
    df_list = []
    total_files = 0
    successful_files = 0
    files_by_date = {}
    
    # Tạo danh sách ngày target
    current_date = start_date
    target_dates = set()
    
    while current_date <= end_date:
        date_str = current_date.strftime('%Y.%m.%d')
        target_dates.add(date_str)
        files_by_date[date_str] = set()
        current_date += timedelta(days=1)
    
    print(f"🚀 Đọc và xử lý dữ liệu A3/CPC từ {start_date.strftime('%Y.%m.%d')} đến {end_date.strftime('%Y.%m.%d')}")
    print(f"📊 Tổng cộng {len(target_dates)} ngày cần xử lý")
    
    # BƯỚC 1: ĐỌC DỮ LIỆU THEO KHOẢNG NGÀY
    years_in_range = set()
    for date_str in target_dates:
        year = date_str.split('.')[0]
        years_in_range.add(year)
    
    for year in sorted(years_in_range):
        year_path = os.path.join(root_path, year)
        
        if not os.path.exists(year_path):
            continue
        
        print(f"\n📅 Đang xử lý năm: {year}")
        
        for month in range(1, 13):
            folder_path = os.path.join(year_path, f'{month:02d}')
            
            if not os.path.exists(folder_path):
                continue
            
            for file in os.listdir(folder_path):
                if file.endswith('.xlsx'):
                    if '-A3' not in file and '-CPC' not in file:
                        continue
                        
                    date_match = re.search(r'(\d{4}\.\d{2}\.\d{2})', file)
                    date_part = date_match.group(1) if date_match else ""
                    
                    if date_part not in target_dates:
                        continue
                    
                    total_files += 1
                    file_path = os.path.join(folder_path, file)
                    
                    try:
                        if '-A3' in file:
                            file_type = 'A3'
                        elif '-CPC' in file:
                            file_type = 'CPC'
                        
                        files_by_date[date_part].add(file_type)
                        
                        df_full = pd.read_excel(file_path, header=None)
                        
                        start_row, status_col = None, None
                        found = False
                        
                        for r_idx, row in df_full.iterrows():
                            for c_idx, cell_value in enumerate(row):
                                if str(cell_value).strip() == 'Trạng thái':
                                    start_row = r_idx
                                    status_col = c_idx
                                    found = True
                                    break
                            if found:
                                break
                        
                        if start_row is not None:
                            header_row = df_full.iloc[start_row]
                            
                            end_col = None
                            for c_idx, cell_value in enumerate(header_row):
                                if str(cell_value).strip() == '00:00':
                                    end_col = c_idx
                                    break
                            
                            if end_col is not None:
                                data_cols = list(range(status_col, end_col + 1))
                                actual_header = [str(header_row.iloc[i]).strip() for i in data_cols]
                                df_data = df_full.iloc[start_row + 1:, data_cols].copy()
                                
                                df_data.columns = actual_header
                                df_data.dropna(how='all', inplace=True)
                                df_data.reset_index(drop=True, inplace=True)
                                
                                if len(actual_header) >= len(expected_columns):
                                    df_data = df_data.iloc[:, :len(expected_columns)]
                                    df_data.columns = expected_columns
                                    
                                    df_data.insert(0, 'Ngày', date_part)
                                    df_data.insert(1, 'Loại_File', file_type)
                                    
                                    df_list.append(df_data)
                                    successful_files += 1
                                    
                                    if total_files % 10 == 0:
                                        print(f"    ✅ Đã xử lý {total_files} file...")
                        
                    except Exception as e:
                        print(f"    ❌ Lỗi file {file}: {e}")
    
    # Kiểm tra file thiếu
    check_missing_a3_cpc_files(files_by_date, target_dates)
    
    if not df_list:
        print(f'\n❌ Không đọc được dữ liệu A3/CPC nào trong khoảng ngày đã chọn')
        return pd.DataFrame()
    
    # Gộp dữ liệu thô
    df_raw = pd.concat(df_list, ignore_index=True)
    print(f'\n📊 ĐỌC DỮ LIỆU THÀNH CÔNG:')
    print(f'   - Tổng file A3/CPC kiểm tra: {total_files}')
    print(f'   - File đọc thành công: {successful_files}')
    print(f'   - Tổng số dòng dữ liệu thô: {len(df_raw):,}')
    
    # BƯỚC 2: XỬ LÝ VÀ LỌC DỮ LIỆU
    print(f'\n🔄 ĐANG XỬ LÝ VÀ LỌC DỮ LIỆU...')
    
    df_filtered = df_raw[df_raw['Loại'].isin(['Báo cáo sau vận hành', 'Báo cáo sau vận hành tổng hợp'])]
    print(f'   - Sau khi lọc theo loại báo cáo: {len(df_filtered):,} dòng')
    
    if df_filtered.empty:
        print(f'❌ Không có dữ liệu sau khi lọc!')
        return pd.DataFrame()
    
    df_processed = df_filtered.drop(columns=['Code', 'Loại', 'Trạng thái']).reset_index(drop=True)
    print(f'   - Sau khi loại bỏ cột không cần thiết: {len(df_processed.columns)} cột')
    
    # Sắp xếp lại cột
    if 'Ngày' in df_processed.columns and '00:30' in df_processed.columns:
        cols = df_processed.columns.tolist()
        cols.remove('Ngày')
        time_30_index = cols.index('00:30')
        cols.insert(time_30_index, 'Ngày')
        df_processed = df_processed[cols]
        print(f'   - Đã sắp xếp lại thứ tự cột')
    
    # Kết quả cuối cùng
    print(f'\n🎉 HOÀN THÀNH XỬ LÝ DỮ LIỆU:')
    print(f'   - Tổng số dòng cuối cùng: {len(df_processed):,}')
    print(f'   - Số cột cuối cùng: {len(df_processed.columns)}')
    
    return df_processed

def check_missing_a3_cpc_files(files_by_date, target_dates=None):
    """Kiểm tra file A3 và CPC thiếu cho từng ngày"""
    
    required_files = {'A3', 'CPC'}
    missing_reports = []
    complete_days = 0
    
    dates_to_check = target_dates if target_dates else files_by_date.keys()
    
    for date in sorted(dates_to_check):
        available_files = files_by_date.get(date, set())
        missing_files = required_files - available_files
        
        if not missing_files:
            complete_days += 1
        elif len(missing_files) == 2:
            missing_reports.append(f"   📅 {date}: ❌ Thiếu cả A3 và CPC")
        else:
            missing_file = list(missing_files)[0]
            missing_reports.append(f"   📅 {date}: ⚠️ Thiếu file {missing_file}")
    
    print(f'\n📋 BÁO CÁO FILE A3/CPC THIẾU:')
    total_days = len(dates_to_check)
    missing_days = total_days - complete_days
    
    print(f'   - Tổng số ngày kiểm tra: {total_days}')
    print(f'   - Ngày đầy đủ file (A3 + CPC): {complete_days}')
    print(f'   - Ngày thiếu file: {missing_days}')
    
    if missing_reports:
        print(f'\n⚠️ CHI TIẾT NGÀY THIẾU FILE:')
        for report in missing_reports:
            print(report)
    else:
        print(f'\n✅ Tất cả ngày đều có đầy đủ file A3 và CPC!')

# ===== SỬ DỤNG LUỒNG THỐNG NHẤT =====

print("🔥 LUỒNG THỐNG NHẤT: Đọc và xử lý dữ liệu A3/CPC!")
user_confirm = input("Bạn có muốn đọc tất cả dữ liệu A3/CPC? (y/n): ")

if user_confirm.lower() == 'y':
    # Đọc và xử lý tất cả dữ liệu
    df_trung_final = read_and_process_a3_cpc_data()
    
    if not df_trung_final.empty:
        print(f"\n✅ Dữ liệu đã sẵn sàng sử dụng: df_trung_final")
        print(f"Shape: {df_trung_final.shape}")

else:
    print("\n📅 Nhập khoảng ngày cần đọc và xử lý dữ liệu A3/CPC:")
    
    # Nhập ngày bắt đầu
    while True:
        start_date_input = input("Ngày bắt đầu (YYYYMMDD): ").strip()
        try:
            datetime.strptime(start_date_input, '%Y%m%d')
            break
        except ValueError:
            print("❌ Định dạng không đúng! Vui lòng nhập theo format YYYYMMDD")
    
    # Nhập ngày kết thúc
    while True:
        end_date_input = input("Ngày kết thúc (YYYYMMDD): ").strip()
        try:
            end_date = datetime.strptime(end_date_input, '%Y%m%d')
            start_date = datetime.strptime(start_date_input, '%Y%m%d')
            
            if start_date <= end_date:
                break
            else:
                print("❌ Ngày kết thúc phải lớn hơn hoặc bằng ngày bắt đầu!")
        except ValueError:
            print("❌ Định dạng không đúng! Vui lòng nhập theo format YYYYMMDD")
    
    # Đọc và xử lý dữ liệu theo khoảng ngày
    df_trung_final = read_and_process_a3_cpc_by_date_range(start_date_input, end_date_input)
    
    if not df_trung_final.empty:
        print(f"\n✅ Dữ liệu đã sẵn sàng sử dụng: df_trung_final")
        print(f"Shape: {df_trung_final.shape}")

print("\n🎯 Hoàn thành luồng thống nhất!")

🔥 LUỒNG THỐNG NHẤT: Đọc và xử lý dữ liệu A3/CPC!
🚀 Bắt đầu đọc và xử lý dữ liệu A3/CPC từ: P:/15. Hệ thống DGMS/Daily Report/Trung/BCSVH_xlsx

📅 Đang xử lý năm: 2024
  📂 Tháng 08...
    ✅ Đã xử lý 20 file...
    ✅ Hoàn thành tháng 08: 39 file
  📂 Tháng 09...
    ✅ Đã xử lý 40 file...
    ✅ Đã xử lý 60 file...
    ✅ Đã xử lý 80 file...
    ✅ Hoàn thành tháng 09: 60 file
  📂 Tháng 10...
    ✅ Đã xử lý 100 file...
    ✅ Đã xử lý 120 file...
    ✅ Đã xử lý 140 file...
    ✅ Đã xử lý 160 file...
    ✅ Hoàn thành tháng 10: 62 file
  📂 Tháng 11...
    ✅ Đã xử lý 180 file...
    ✅ Đã xử lý 200 file...
    ✅ Đã xử lý 220 file...
    ✅ Hoàn thành tháng 11: 60 file
  📂 Tháng 12...
    ✅ Đã xử lý 240 file...
    ✅ Đã xử lý 260 file...
    ✅ Đã xử lý 280 file...
    ✅ Hoàn thành tháng 12: 62 file

📅 Đang xử lý năm: 2025
  📂 Tháng 01...
    ✅ Đã xử lý 300 file...
    ✅ Đã xử lý 320 file...
    ✅ Đã xử lý 340 file...
    ✅ Hoàn thành tháng 01: 62 file
  📂 Tháng 02...
    ✅ Đã xử lý 360 file...
    ✅ 

In [4]:
import os

# Tạo thư mục nếu chưa tồn tại
output_dir = r"D:\OneDrive\0. Code\TDN\data\interim\Trung"
os.makedirs(output_dir, exist_ok=True)

# Lưu dữ liệu ra file parquet
if 'df_trung_final' in locals() and not df_trung_final.empty:
    # Xử lý cột "Nhà máy" để tránh lỗi mixed types
    print("🔧 Đang xử lý kiểu dữ liệu...")
    
    # Kiểm tra dữ liệu trong cột "Nhà máy"
    print(f"Kiểu dữ liệu của cột 'Nhà máy': {df_trung_final['Nhà máy'].dtype}")
    print(f"Các giá trị unique trong cột 'Nhà máy': {df_trung_final['Nhà máy'].unique()[:10]}")
    
    # Tạo bản copy để xử lý
    df_save = df_trung_final.copy()
    
    # Chuyển đổi cột "Nhà máy" về kiểu string thống nhất
    df_save['Nhà máy'] = df_save['Nhà máy'].astype(str)
    
    # Xử lý các cột khác có thể có vấn đề tương tự
    object_columns = df_save.select_dtypes(include=['object']).columns
    for col in object_columns:
        try:
            df_save[col] = df_save[col].astype(str)
        except:
            print(f"⚠️ Không thể chuyển đổi cột {col}")
    
    # Lưu file
    output_file = os.path.join(output_dir, "df_trung_final.parquet")
    df_save.to_parquet(output_file, index=False)
    print(f"✅ Đã lưu dữ liệu ra file: {output_file}")
    print(f"📊 Kích thước file: {os.path.getsize(output_file) / (1024*1024):.2f} MB")
    print(f"📋 Shape: {df_save.shape}")
else:
    print("❌ Không có dữ liệu để lưu!")

🔧 Đang xử lý kiểu dữ liệu...
Kiểu dữ liệu của cột 'Nhà máy': object
Các giá trị unique trong cột 'Nhà máy': ['Vĩnh Sơn 5' 'ĐG Phương Mai 1' 'NƯỚC LƯƠNG' 'ĐG Phương Mai 3' 'Trà Xom'
 'Ken Lút Hạ' 'Hòa Phú' 'ĐG Tây Nguyên' 'MT Jang Pông' 'MT BMT']
✅ Đã lưu dữ liệu ra file: D:\Khuê\Khue\TDN\data\interim\Trung\df_trung_final.parquet
📊 Kích thước file: 2.67 MB
📋 Shape: (50043, 54)


In [5]:
# Đọc dữ liệu tham chiếu
df_thamchieu = pd.read_excel(r"D:\Khuê\Khue\TDN\data\raw\DS_TTT_TDN.xlsx")
print(f"✅ Đọc thành công df_thamchieu - Shape: {df_thamchieu.shape}")

✅ Đọc thành công df_thamchieu - Shape: (537, 20)


In [6]:
df_thamchieu

,ID_NM,DIEU_DO,TCTDL,DIEN_LUC,TENNHAMAY,TEN_NM,LOAI_HINH,QUYEN_DIEUKHIEN,CS_DAT,A_TB_NAM,U_DAUNOI,MIEN,TINH,HUYEN,KINHDO,VIDO,BACTHANGSONG,TD_THAMCHIEU,HIEU_LUC,DVPD
0,10997,A1,NPC,PC_BACKAN,Nặm Cắt,NAM_CAT,1,PC_BACKAN,3.2,0.000,35,Bắc,Bắc Kạn,Bạch Thông,105.789306,22.192518,SONG_HONG,TUYEN_QUANG,1,NaN
1,10998,A1,NPC,PC_BACKAN,Pác Cáp,PAC_CAP,1,PC_BACKAN,6.0,0.000,0,NaN,NaN,NaN,106.127303,22.210818,SONG_HONG,TUYEN_QUANG,1,NaN
2,10995,A1,NPC,PC_BACKAN,Tà Làng,TA_LANG,1,PC_BACKAN,4.5,0.000,0,NaN,NaN,NaN,105.721185,22.303931,SONG_HONG,TUYEN_QUANG,1,NaN
3,11193,A1,NPC,PC_BACKAN,THÁC GIỀNG 1,THAC_GIENG 1,1,PC_BACKAN,5.5,0.000,0,NaN,NaN,NaN,105.893735,22.095786,SONG_HONG,TUYEN_QUANG,1,NaN
4,11191,A1,NPC,PC_BACKAN,Thượng Ân,THUONG_AN,1,PC_BACKAN,2.4,0.000,0,NaN,NaN,NaN,106.003945,22.511839,SONG_HONG,TUYEN_QUANG,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,11054,A1,NPC,PC_YENBAI,Thào Sa Chải,THAO_SA_CHAI,1,PC_YENBAI,6.5,22.684,35,Bắc,Yên Bái,Mù Cang Chải,104.268669,21.838929,SONG_HONG,BAN_CHAT,0,NaN
533,10830,A1,NPC,PC_YENBAI,Trạm Tấu,TRAM_TAU,1,A1,30.0,0.000,0,NaN,NaN,NaN,104.442036,21.498660,SONG_HONG,BAN_CHAT,1,NaN
534,10840,A1,NPC,PC_YENBAI,Vực Tuần,VUC_TUAN,1,PC_YENBAI,5.0,0.000,0,NaN,NaN,NaN,104.569095,21.416027,SONG_HONG,BAN_CHAT,1,NaN
535,10837,A1,NPC,PC_YENBAI,Nậm Tục,NAM_TUC,1,PC_YENBAI,3.0,0.000,0,NaN,NaN,NaN,104.447998,21.608201,SONG_HONG,BAN_CHAT,1,NaN


In [7]:
df_trung_final

,Loại_File,Nhà máy,P.đặt,Q.ĐK,Điện lực,Ngày,00:30,01:00,01:30,02:00,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,00:00
0,A3,Vĩnh Sơn 5,28,A3,PC_BINHDINH,2024.08.12,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A3,ĐG Phương Mai 1,26.4,A3,PC_BINHDINH,2024.08.12,6.3,10.9,5.8,4.9,...,0,0,0.1,0.2,0.7,0.9,2.1,0,1.4,1.6
2,A3,NƯỚC LƯƠNG,22,A3,PC_BINHDINH,2024.08.12,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A3,ĐG Phương Mai 3,20.79,A3,PC_BINHDINH,2024.08.12,11.2,10.7,5.9,4.2,...,0.2,0.2,0.2,0.4,1.2,1.3,1.3,0.8,3.6,3.2
4,A3,Trà Xom,20,A3,PC_BINHDINH,2024.08.12,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50038,CPC,Khe Giông,9,PC_QUANGTRI,PC_QUANGTRI,2025.05.31,0,0,0,0,...,4.5,4.5,0,0,0,0,0,0,0,0
50039,CPC,Khe Nghi,8,PC_QUANGTRI,PC_QUANGTRI,2025.05.31,0,0,0,0,...,8,8,5,2.5,3,0,0,0,0,0
50040,CPC,Đăk Rông 3,8,PC_QUANGTRI,PC_QUANGTRI,2025.05.31,4,4,0,0,...,8,8,4,4,4,4,4,4,0,0
50041,CPC,Hạ Rào Quán,6.4,PC_QUANGTRI,PC_QUANGTRI,2025.05.31,5.9,5.9,5.9,5.4,...,5.9,5.9,5.9,5.9,5.9,5.9,5.9,5.9,5.9,5.9


In [8]:
# Merge dữ liệu tham chiếu vào df_trung_final
print("🔄 Đang merge dữ liệu tham chiếu...")

# Kiểm tra các cột key trước khi merge
print(f"📊 Số lượng nhà máy unique trong df_trung_final: {df_trung_final['Nhà máy'].nunique()}")
print(f"📊 Số lượng nhà máy unique trong df_thamchieu: {df_thamchieu['TENNHAMAY'].nunique()}")

# Thực hiện merge với left join để giữ tất cả dữ liệu từ df_trung_final
df_merged = df_trung_final.merge(
    df_thamchieu[['TENNHAMAY', 'TD_THAMCHIEU']], 
    left_on='Nhà máy', 
    right_on='TENNHAMAY', 
    how='left'
)

print(f"✅ Merge hoàn thành - Shape: {df_merged.shape}")
print(f"📋 Số dòng trước merge: {df_trung_final.shape[0]}")
print(f"📋 Số dòng sau merge: {df_merged.shape[0]}")

# Kiểm tra số lượng giá trị null trong cột TD_THAMCHIEU
null_count = df_merged['TD_THAMCHIEU'].isnull().sum()
print(f"⚠️ Số dòng không có thông tin tham chiếu: {null_count}")

# Hiển thị một số dòng mẫu
print("\n📝 Dữ liệu sau merge:")
print(df_merged[['Nhà máy', 'TENNHAMAY', 'TD_THAMCHIEU']].head())

# Cập nhật df_trung_final
df_trung_final = df_merged.drop(columns=['TENNHAMAY'])
print(f"🔄 Đã cập nhật df_trung_final - Shape: {df_trung_final.shape}")


🔄 Đang merge dữ liệu tham chiếu...
📊 Số lượng nhà máy unique trong df_trung_final: 210
📊 Số lượng nhà máy unique trong df_thamchieu: 534
✅ Merge hoàn thành - Shape: (50043, 56)
📋 Số dòng trước merge: 50043
📋 Số dòng sau merge: 50043
⚠️ Số dòng không có thông tin tham chiếu: 8973

📝 Dữ liệu sau merge:
           Nhà máy   TENNHAMAY TD_THAMCHIEU
0       Vĩnh Sơn 5  Vĩnh Sơn 5     VINH_SON
1  ĐG Phương Mai 1         NaN          NaN
2       NƯỚC LƯƠNG         NaN          NaN
3  ĐG Phương Mai 3         NaN          NaN
4          Trà Xom     Trà Xom     VINH_SON
🔄 Đã cập nhật df_trung_final - Shape: (50043, 55)


In [11]:
# Xuất dữ liệu ra file Excel vào thư mục Trung
print("📤 Đang xuất dữ liệu ra file Excel...")

# Tạo tên file với date
from datetime import datetime
date_str = datetime.now().strftime("%Y%m%d")
output_file = f"D:\Khuê\Khue\TDN\data\interim\Trung/DGMS_data_{date_str}.xlsx"

# Xuất df_trung_final ra Excel
df_trung_final.to_excel(output_file, index=False)

print(f"✅ Xuất file thành công: {output_file}")
print(f"📊 Dữ liệu đã được lưu với {df_trung_final.shape[0]} dòng và {df_trung_final.shape[1]} cột")

📤 Đang xuất dữ liệu ra file Excel...
✅ Xuất file thành công: D:\Khuê\Khue\TDN\data\interim\Trung/DGMS_data_20250616.xlsx
📊 Dữ liệu đã được lưu với 50043 dòng và 55 cột


In [ ]:
import pandas as pd
import warnings
import os
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import calendar
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

# Đọc file Sanluong
sanluong_path = r"D:\OneDrive\0. Code\TDN\data\interim\Trung\DGMS_data_20250616.xlsx"
df_sanluong = pd.read_excel(sanluong_path, engine='openpyxl')
df_sanluong

,Loại_File,Nhà máy,P.đặt,Q.ĐK,Điện lực,TD_THAMCHIEU,Ngày,00:30,01:00,01:30,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,00:00
0,A3,Vĩnh Sơn 5,28.0,A3,PC_BINHDINH,VINH_SON,2024.08.12,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A3,NƯỚC LƯƠNG,22.0,A3,PC_BINHDINH,DAK_DRINH,2024.08.12,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A3,Trà Xom,20.0,A3,PC_BINHDINH,VINH_SON,2024.08.12,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
3,A3,Ken Lút Hạ,6.0,A3,PC_BINHDINH,VINH_SON,2024.08.12,0.0,0.0,0.0,...,5.4,5.4,5.0,4.2,0.0,0.0,0.0,0.0,0.0,0.0
4,A3,Hòa Phú,29.0,A3,PC_DAKLAK,BUON_TUA_SRAH,2024.08.12,26.3,26.3,25.2,...,25.9,25.9,19.5,19.5,19.5,19.5,26.0,26.0,25.8,25.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49464,CPC,Khe Giông,9.0,PC_QUANGTRI,PC_QUANGTRI,QUANGTRI,2025.05.31,0.0,0.0,0.0,...,4.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49465,CPC,Khe Nghi,8.0,PC_QUANGTRI,PC_QUANGTRI,QUANGTRI,2025.05.31,0.0,0.0,0.0,...,8.0,8.0,5.0,2.5,3.0,0.0,0.0,0.0,0.0,0.0
49466,CPC,Đăk Rông 3,8.0,PC_QUANGTRI,PC_QUANGTRI,QUANGTRI,2025.05.31,4.0,4.0,0.0,...,8.0,8.0,4.0,4.0,4.0,4.0,4.0,4.0,0.0,0.0
49467,CPC,Hạ Rào Quán,6.4,PC_QUANGTRI,PC_QUANGTRI,QUANGTRI,2025.05.31,5.9,5.9,5.9,...,5.9,5.9,5.9,5.9,5.9,5.9,5.9,5.9,5.9,5.9


In [20]:
# Kiểm tra lại các dòng còn thiếu giá trị trong cột "P.Đặt"
missing_pdat = df_sanluong[df_sanluong['P.đặt'].isnull()]
# Kiểm tra các dòng còn thiếu giá trị trong cột "P.Đặt"
missing_pdat = df_sanluong[df_sanluong['P.đặt'].isnull()]
if not missing_pdat.empty:
    print(f"Còn {len(missing_pdat)} dòng thiếu giá trị trong cột 'P.đặt':")
    print(missing_pdat[['Nhà máy', 'TD_THAMCHIEU', 'P.đặt']].drop_duplicates())
else:
    print("Không còn dòng nào thiếu giá trị trong cột 'P.đặt'")

Không còn dòng nào thiếu giá trị trong cột 'P.đặt'


In [21]:
df_sanluong['SL'] = df_sanluong.loc[:, '00:30':'00:00'].sum(axis=1) / 2
df_dgms_sanluong = df_sanluong[['TD_THAMCHIEU', 'Ngày', 'SL', 'P.đặt']].groupby(['TD_THAMCHIEU', 'Ngày'], as_index=False).sum()
df_dgms_sanluong

,TD_THAMCHIEU,Ngày,SL,P.đặt
0,BUON_TUA_SRAH,2024.08.12,547.000,167.67
1,BUON_TUA_SRAH,2024.08.13,492.100,167.67
2,BUON_TUA_SRAH,2024.08.14,566.200,167.67
3,BUON_TUA_SRAH,2024.08.15,561.100,167.67
4,BUON_TUA_SRAH,2024.08.16,562.900,167.67
...,...,...,...,...
3016,VINH_SON,2025.05.27,751.645,94.80
3017,VINH_SON,2025.05.28,851.935,94.80
3018,VINH_SON,2025.05.29,1081.920,94.80
3019,VINH_SON,2025.05.30,967.930,94.80


In [22]:
# Chuyển cột 'SL' và 'P.đặt' sang kiểu số
df_dgms_sanluong['SL'] = pd.to_numeric(df_dgms_sanluong['SL'], errors='coerce')
df_dgms_sanluong['P.đặt'] = pd.to_numeric(df_dgms_sanluong['P.đặt'], errors='coerce')
# Chuyển giá trị cột 'SL' từ MWh sang triệu kWh
df_dgms_sanluong['SL'] = df_dgms_sanluong['SL'] / 1000  # Chia cho 1000 để chuyển từ MWh sang triệu kWh

In [23]:
unique_td_thamchieu = df_dgms_sanluong['TD_THAMCHIEU'].unique()
print(f"Số lượng giá trị unique trong cột TD_THAMCHIEU: {len(unique_td_thamchieu)}")
print("\nCác giá trị unique trong cột TD_THAMCHIEU:")
for value in sorted(unique_td_thamchieu):
    print(f"- {value}")


Số lượng giá trị unique trong cột TD_THAMCHIEU: 11

Các giá trị unique trong cột TD_THAMCHIEU:
- BUON_TUA_SRAH
- DAK_DRINH
- DAK_RTIH
- HUONG_DIEN
- IALY
- QUANGTRI
- SONG_BA_HA
- SONG_BUNG_2
- SONG_TRANH_2
- THUONG_KONTUM
- VINH_SON


In [24]:
df_Qve = pd.read_csv(r"D:\OneDrive\0. Code\TDN\data\interim\Trung\Q_ve_trung.csv")
df_Qve['NGAY_VANHANH'] = pd.to_datetime(df_Qve['NGAY_VANHANH']).dt.strftime('%Y.%m.%d')
df_Qve

,TENTAT_NM,NGAY_VANHANH,SANLUONG_NGAY,QVE,QMAY,QXA,HTL
0,BUON_TUA_SRAH,2021.01.01,658.455,79.00,54.46,0.0,487.068
1,BUON_TUA_SRAH,2021.01.02,986.733,73.63,81.62,0.0,487.049
2,BUON_TUA_SRAH,2021.01.03,1022.145,69.63,84.71,0.0,487.014
3,BUON_TUA_SRAH,2021.01.04,1559.998,70.21,129.42,0.0,486.876
4,BUON_TUA_SRAH,2021.01.05,874.818,76.38,72.21,0.0,486.886
...,...,...,...,...,...,...,...
21159,DAK_DRINH,2025.06.13,449.000,49.78,7.78,0.0,403.440
21160,DAK_DRINH,2025.06.14,1030.000,33.05,18.58,0.0,403.580
21161,DAK_DRINH,2025.06.15,662.000,28.23,13.26,0.0,403.720
21162,DAK_DRINH,2025.06.16,558.000,27.00,8.40,0.0,403.900


In [25]:
# chuyển giá trị cột 'Ngày' trong df_dgms_sanluong và 'NGAY_VANHANH' trong df_Qve về định dạng datetime
df_dgms_sanluong['Ngày'] = pd.to_datetime(df_dgms_sanluong['Ngày'])
df_Qve['NGAY_VANHANH'] = pd.to_datetime(df_Qve['NGAY_VANHANH'])

In [26]:
# Chuẩn hóa định dạng cột TD_THAMCHIEU và TENTAT_NM để đảm bảo merge chính xác
df_dgms_sanluong['TD_THAMCHIEU'] = df_dgms_sanluong['TD_THAMCHIEU'].astype(str).str.strip()
df_Qve['TENTAT_NM'] = df_Qve['TENTAT_NM'].astype(str).str.strip()

# Kiểm tra các giá trị unique để đảm bảo mapping đúng
print("Các giá trị unique trong TD_THAMCHIEU:", df_dgms_sanluong['TD_THAMCHIEU'].unique())
print("Các giá trị unique trong TENTAT_NM:", df_Qve['TENTAT_NM'].unique())


Các giá trị unique trong TD_THAMCHIEU: ['BUON_TUA_SRAH' 'DAK_DRINH' 'DAK_RTIH' 'HUONG_DIEN' 'IALY' 'QUANGTRI'
 'SONG_BA_HA' 'SONG_BUNG_2' 'SONG_TRANH_2' 'THUONG_KONTUM' 'VINH_SON']
Các giá trị unique trong TENTAT_NM: ['BUON_TUA_SRAH' 'HUONG_DIEN' 'IALY' 'QUANGTRI' 'SONG_BA_HA' 'SONG_BUNG_2'
 'SONG_TRANH_2' 'THUONG_KONTUM' 'VINH_SON' 'DAK_RTIH' 'DAK_DRINH']


In [27]:
# Merge dữ liệu QVE từ df_Qve vào df_dgms_sanluong
df_dgms_sanluong_merge = df_dgms_sanluong.merge(
    df_Qve[['TENTAT_NM', 'NGAY_VANHANH', 'QVE']], 
    left_on=['TD_THAMCHIEU', 'Ngày'], 
    right_on=['TENTAT_NM', 'NGAY_VANHANH'], 
    how='left'
)

# Xóa các cột trung gian không cần thiết
df_dgms_sanluong_merge = df_dgms_sanluong_merge.drop(['TENTAT_NM', 'NGAY_VANHANH'], axis=1)

# Hiển thị kết quả
df_dgms_sanluong_merge


,TD_THAMCHIEU,Ngày,SL,P.đặt,QVE
0,BUON_TUA_SRAH,2024-08-12,0.547000,167.67,90.54
1,BUON_TUA_SRAH,2024-08-13,0.492100,167.67,92.54
2,BUON_TUA_SRAH,2024-08-14,0.566200,167.67,96.33
3,BUON_TUA_SRAH,2024-08-15,0.561100,167.67,103.71
4,BUON_TUA_SRAH,2024-08-16,0.562900,167.67,101.79
...,...,...,...,...,...
3602,VINH_SON,2025-05-29,1.081920,94.80,1.80
3603,VINH_SON,2025-05-30,0.967930,94.80,1.07
3604,VINH_SON,2025-05-30,0.967930,94.80,2.19
3605,VINH_SON,2025-05-31,0.911945,94.80,0.80


In [28]:
# export df_dgms_sanluong_merge
df_dgms_sanluong_merge.to_excel(r"D:\OneDrive\0. Code\TDN\data\interim\Trung\DGMS_BCSVH_sanluong_trung.xlsx", index=False)